In [98]:
# initialization and random seed set

import os
import scipy
import torch
import random
import sklearn
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from CBIG_model_pytorch import dnn_4l, dnn_5l
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings("ignore")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Step 1. Load Data

In [13]:
# Age와 관련한 파일 
subejct_df = pd.read_csv('for_reproduce/MMP_HCP_753_Age_list.csv', index_col = 0)
# 581개의 Phenotype과 관련한 파일 
phenotype = pd.read_csv('for_reproduce/unrestricted_sophiafrangou_7_11_2021_22_0_0.csv', index_col = 0)

# 이전의 파일들과는 다르게 Fluid Inteligence를 가장 뒤로 두었다. 
pheno_list = ['PicSeq_Unadj', 'CardSort_Unadj', 'Flanker_Unadj',
 'ReadEng_Unadj', 'PicVocab_Unadj', 'ProcSpeed_Unadj',
 'DDisc_AUC_40K', 'VSPLOT_TC', 'SCPT_SEN', 'SCPT_SPEC',
 'IWRD_TOT', 'ListSort_Unadj', 'MMSE_Score', 'PSQI_Score', 'Endurance_Unadj',
 'GaitSpeed_Comp', 'Dexterity_Unadj', 'Strength_Unadj', 'Odor_Unadj', 'PainInterf_Tscore',
 'Taste_Unadj', 'Mars_Final', 'Emotion_Task_Face_Acc',
 'Language_Task_Math_Avg_Difficulty_Level', 'Language_Task_Story_Avg_Difficulty_Level',
 'Relational_Task_Acc', 'Social_Task_Perc_Random',
 'Social_Task_Perc_TOM', 'WM_Task_Acc', 'NEOFAC_A', 'NEOFAC_O',
 'NEOFAC_C', 'NEOFAC_N', 'NEOFAC_E', 'ER40_CR', 'ER40ANG', 'ER40FEAR', 'ER40HAP', 'ER40NOE',
 'ER40SAD', 'AngAffect_Unadj', 'AngHostil_Unadj', 'AngAggr_Unadj', 'FearAffect_Unadj',
 'FearSomat_Unadj', 'Sadness_Unadj', 'LifeSatisf_Unadj',
 'MeanPurp_Unadj', 'PosAffect_Unadj', 'Friendship_Unadj', 'Loneliness_Unadj',
 'PercHostil_Unadj', 'PercReject_Unadj', 'EmotSupp_Unadj', 'InstruSupp_Unadj',
 'PercStress_Unadj', 'SelfEff_Unadj', 'PMAT24_A_CR']

# Reference 논문에서 사용되는 최종적인 58개의 Phenotype만 골라낸다.
phenotype = phenotype.loc[:, pheno_list]
phenotype.head()

,PicSeq_Unadj,CardSort_Unadj,Flanker_Unadj,ReadEng_Unadj,PicVocab_Unadj,ProcSpeed_Unadj,DDisc_AUC_40K,VSPLOT_TC,SCPT_SEN,SCPT_SPEC,...,PosAffect_Unadj,Friendship_Unadj,Loneliness_Unadj,PercHostil_Unadj,PercReject_Unadj,EmotSupp_Unadj,InstruSupp_Unadj,PercStress_Unadj,SelfEff_Unadj,PMAT24_A_CR
Subject,,,,,,,,,,,,,,,,,,,,,
100004,118.70,103.17,121.97,109.9589,111.1118,92.72,0.067448,9.0,0.8833,0.9917,...,36.2,27.6,73.0,60.0,70.4,29.3,34.8,55.4,39.6,19.0
100206,125.07,119.14,130.42,113.5460,119.8914,138.72,0.050000,12.0,0.9667,0.9583,...,48.4,45.7,63.8,63.8,64.8,43.9,46.0,57.8,44.0,20.0
100307,110.61,123.75,112.56,111.0700,107.2000,122.65,0.311459,9.0,0.9833,0.9583,...,46.2,62.0,53.7,52.7,54.1,45.0,52.3,57.9,37.5,17.0
100408,125.71,111.14,121.18,131.8100,134.2400,107.08,0.421354,17.0,0.8667,0.9167,...,48.4,52.9,51.9,52.2,50.1,50.1,51.1,46.8,56.4,7.0
100610,109.04,129.84,126.53,141.3166,140.8151,111.11,0.868750,18.0,1.0000,0.9750,...,55.1,44.9,53.5,38.5,54.1,56.8,48.4,37.8,53.2,23.0


In [14]:
# 각 subject의 번호 (phenotype에서 논문에서 사용한 subject 들에 대해서만 Row로 추출하기 위해)
subject_list = subejct_df.index.tolist()
phenotype = phenotype.loc[subject_list, :]
phenotype.shape

(753, 58)

In [15]:
age = subejct_df[['Age']]
phenotype = pd.concat([age, phenotype], axis = 1)
phenotype

,Age,PicSeq_Unadj,CardSort_Unadj,Flanker_Unadj,ReadEng_Unadj,PicVocab_Unadj,ProcSpeed_Unadj,DDisc_AUC_40K,VSPLOT_TC,SCPT_SEN,...,PosAffect_Unadj,Friendship_Unadj,Loneliness_Unadj,PercHostil_Unadj,PercReject_Unadj,EmotSupp_Unadj,InstruSupp_Unadj,PercStress_Unadj,SelfEff_Unadj,PMAT24_A_CR
Subject,,,,,,,,,,,,,,,,,,,,,
100206,27,125.07,119.14,130.42,113.5460,119.89140,138.72,0.050000,12.0,0.9667,...,48.4,45.7,63.8,63.8,64.8,43.9,46.0,57.8,44.0,20.0
100307,27,110.61,123.75,112.56,111.0700,107.20000,122.65,0.311459,9.0,0.9833,...,46.2,62.0,53.7,52.7,54.1,45.0,52.3,57.9,37.5,17.0
100408,33,125.71,111.14,121.18,131.8100,134.24000,107.08,0.421354,17.0,0.8667,...,48.4,52.9,51.9,52.2,50.1,50.1,51.1,46.8,56.4,7.0
101006,35,84.68,105.19,101.85,113.5374,95.42348,90.59,0.783073,20.0,0.9833,...,42.0,57.1,50.3,46.1,46.5,62.5,43.3,48.8,38.8,11.0
101107,22,105.60,119.76,107.04,119.2434,119.84590,112.27,0.584375,16.0,0.9167,...,59.2,66.5,52.0,63.8,43.7,62.5,32.6,51.9,68.4,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992673,33,96.55,94.96,94.87,117.9919,118.44630,104.06,0.791667,11.0,0.8833,...,50.2,49.8,50.3,47.8,40.7,62.5,46.7,41.0,49.3,21.0
992774,35,89.89,126.37,124.64,108.6900,106.09000,114.31,0.019531,9.0,0.9333,...,48.4,47.5,49.6,52.5,50.6,51.8,46.7,46.7,54.7,14.0
993675,29,108.89,120.35,116.43,129.4080,135.66530,113.40,0.938281,21.0,0.9833,...,46.2,50.7,53.7,53.9,54.1,44.8,45.9,49.8,45.9,21.0


In [16]:
cort_thick = np.load('for_reproduce/smri/cortical_thick.npy')
cort_area = np.load('for_reproduce/smri/cortical_area.npy')
cort_vol = np.load('for_reproduce/smri/cortical_vol.npy')


tbss_ad = np.load('./data/for_reproduce/dmri/tbss/tbss_ad.npy')
tbss_fa = np.load('./data/for_reproduce/dmri/tbss/tbss_fa.npy')
tbss_icvf = np.load('./data/for_reproduce/dmri/tbss/tbss_icvf.npy')
tbss_isovf = np.load('./data/for_reproduce/dmri/tbss/tbss_isovf.npy')
tbss_md = np.load('./data/for_reproduce/dmri/tbss/tbss_md.npy')
tbss_od = np.load('./data/for_reproduce/dmri/tbss/tbss_od.npy')
tbss_rd = np.load('./data/for_reproduce/dmri/tbss/tbss_rd.npy')

tractography_ad = np.load('./data/for_reproduce/dmri/tracto/tracto_ad.npy')
tractography_fa = np.load('./data/for_reproduce/dmri/tracto/tracto_fa.npy')
tractography_icvf = np.load('./data/for_reproduce/dmri/tracto/tracto_icvf.npy')
tractography_isovf = np.load('./data/for_reproduce/dmri/tracto/tracto_isovf.npy')
tractography_md = np.load('./data/for_reproduce/dmri/tracto/tracto_md.npy')
tractography_od = np.load('./data/for_reproduce/dmri/tracto/tracto_od.npy')
tractography_rd = np.load('./data/for_reproduce/dmri/tracto/tracto_rd.npy')
tractography_fss = np.load('./data/for_reproduce/dmri/tracto/tracto_fss.npy')
tractography_fssl = np.load('./data/for_reproduce/dmri/tracto/tracto_fssl.npy')


f_gamb = np.load('./data/for_reproduce/fmri/f_gamb.npy')
f_lang = np.load('./data/for_reproduce/fmri/f_lang.npy')
f_motor = np.load('./data/for_reproduce/fmri/f_motor.npy')
f_rs = np.load('./data/for_reproduce/fmri/f_rs.npy')
f_social = np.load('./data/for_reproduce/fmri/f_social.npy')
f_wm = np.load('./data/for_reproduce/fmri/f_wm.npy')

In [21]:
# npy 파일은 Row가 각 ROI를 의미하고, Column이 Subject를 의미한다. 
# 따라서, 이것을 통상적으로 우리가 사용하는 방식이 되기 위해서는 Transpose하는 과정을 거쳐야 한다. 
print(cort_thick.shape)
print(tbss_ad.shape)
print(tractography_ad.shape)
print(f_gamb.shape)

(400, 753)
(133053, 753)
(79800, 753)
(79800, 753)


In [22]:
gpu = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

cuda


In [143]:
# df           : sMRI, dMRI, fMRI에서 얻은 connectom data 
# phenotype    : reference 논문에서 사용한 58개의 phenotype을 가지고 있는 dataframe 
# subject_list : 데이터로 사용하는 subject number들의 list 
def data_shuffling(df, phenotype, subject_list, seed):
    # df는 npy파일로부터 가져온 데이터 -> 이것을 random하게 shuffle한 것이 meta_x_data가 된다. 
    # phenotype은 위에서 정의한 58개 phenotype + Age (가장 마지막 Column : Fluid Inteligence) -> 이것을 random하게 shuffle한 것이 meta_y_data가 된다. 
    random.seed(seed)
    df = pd.DataFrame(df)
    df = df.T
    df.index = subject_list
    
    # subject list를 무작위로 섞는다
    random.shuffle(subject_list)
    # 섞인 순서를 가지고 있는 subject list대로 다시 data frame과 phenotype이 섞인다. 
    df = df.reindex(subject_list)
    phenotype = phenotype.reindex(subject_list)
    
    # df, phenotype이 다시 ndarray의 형태로 변환 
    shuffled_df = df.to_numpy()
    shuffled_phenotype = phenotype.to_numpy()
    
    return shuffled_df, shuffled_phenotype


### Scaling 및 Pre-processing

In [144]:
# def dataset_preprocessing(shuffled_df, shuffled_phenotype, k_num, seed):
def data_split_scaling(shuffled_df, shuffled_phenotype, k_num, seed):
    
    # 20%로 해서 가장 크게 Train meta set과 Test meta set을 나눈다. 
    x_train, x_test, y_train, y_test = train_test_split(shuffled_df, shuffled_phenotype, test_size=0.2, random_state=seed)
    
    # Train meta set을 Train / Validation set split 
    train_x = x_train[:int(x_train.shape[0] * 0.8)]
    val_x = x_train[int(x_train.shape[0] * 0.8):]
    train_pheno = y_train[:int(y_train.shape[0] * 0.8), :]
    val_pheno = y_train[int(y_train.shape[0] * 0.8):, :]
    
    # Train set과 Validation set의 phenotype을 StandardScaling
    train_scaler = StandardScaler()
    train_scaler.fit(train_pheno)
    train_pheno = train_scaler.transform(train_pheno)
    val_pheno = train_scaler.transform(val_pheno)
    
    train_iq = train_pheno[:, -1]
    train_pheno = train_pheno[:, :-1]
    val_iq = val_pheno[:, -1]
    val_pheno = val_pheno[:, :-1]
    
    
    # Test meta set을 K-train / rest_test_meta
    k_sample_x = x_test[:k_num] # k * smri(400) 
    test_x = x_test[k_num:] # rest * smri(400)
    k_sample_pheno = y_test[:k_num]
    test_pheno = y_test[k_num:]
    
    # K-train set과 rest_test_meta set의 phenotype을 StandardScaling
    test_scaler = StandardScaler()
    test_scaler.fit(k_sample_pheno)
    k_sample_pheno = test_scaler.transform(k_sample_pheno)
    test_pheno = test_scaler.transform(test_pheno)
    
    k_sample_iq = k_sample_pheno[:, -1]
    k_sample_pheno = k_sample_pheno[:, :-1]
    test_iq = test_pheno[:, -1]
    test_pheno = test_pheno[:, :-1]

    
    return train_x, val_x, train_pheno, val_pheno, train_iq, val_iq, k_sample_x, test_x, k_sample_pheno, test_pheno, k_sample_iq, test_iq



### Basic KRR

In [145]:
def basic_krr(df, phenotype, subject_list, model, k, iteration=1):
    print(f"Experiment : k={k}, iteration={iteration}")
    
    final_corr_list = []
    
    for seed in range(iteration):
        # Seed
        # X = age_anat_pheno.drop('PMAT24_A_CR', axis=1)
        # y = age_anat_pheno['PMAT24_A_CR']
        shuffled_df, shuffled_phenotype = data_shuffling(df, phenotype, subject_list, seed)
        # train_meta_x, val_meta_x, train_meta_y, val_meta_y, k_sample_x, test_meta_x, k_sample_y, test_meta_y = data_split_scaling(shuffled_df, shuffled_phenotype, k, seed)
        train_x, val_x, train_pheno, val_pheno, train_iq, val_iq, k_sample_x, test_x, k_sample_pheno, test_pheno, k_sample_iq, test_iq = data_split_scaling(shuffled_df, shuffled_phenotype, k, seed)
        
        model_list = [model] * 58
        # 33개의 KRR을 가지고 Age를 가장 잘 예측하는 KRR을 선택 
        best_corr = -100
        best_model_idx = -1
        
        
        # Age를 포함한 58개의 phenotype을 KRR을 사용하여 돌아가면서 k_shot에 대해서 Age prediction을 진행 
        for i, model in enumerate(model_list):
            model.fit(train_x, train_pheno[:, i]) # i번째 phenotype을 load한 npy를 통해 학습
            k_shot_pred = model.predict(k_sample_x) # k-sample들을 예측하여 성능 측정 
            k_shot_corr = np.corrcoef(k_sample_iq, k_shot_pred)[0][1] # k-sample들을 예측한 것과 IQ 간 correlation 계산
            
            if best_corr < k_shot_corr:
                best_model_idx = i
                best_corr = k_shot_corr
                
        best_model = model_list[best_model_idx]
        best_model.fit(train_x, train_pheno[:, best_model_idx])
        best_model_k_shot_metric = np.corrcoef(k_sample_iq, best_model.predict(k_sample_x))
        
        final_pred = best_model.predict(test_x)
        final_corr = round(np.corrcoef(test_iq, final_pred)[0][1], 4)
        final_corr_list.append(final_corr)
        
    
    final_corr_np = np.array(final_corr_list)
    mean = round(np.mean(final_corr_np),4)
    std = round(np.std(final_corr_np),2)
    
    print(f"final correlation mean : {mean}")
    print(f"final correlation std : {std}")
    print(f"final correlation list : {final_corr_list}")
    
    print('\n======================================================================================\n\n')
    
    
    

### Basic DNN

In [167]:
def basic_dnn(df, phenotype, k_num, iteration = 10):
    
    corres = []
    
    # 반복 실험 
    for seed in range(1, iteration+1):
        
        # Random Seed 설정
        print('iter : ', seed)
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)        
        # CUDA를 사용하는 경우 => 이거 반복적으로 동일한 결과가 나오는 지 확인 
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

        
        
        # Anat과 pheno_with_age의 index를 랜덤하게 Shuffling -> Train meta set과 Test meta set으로 분리 
        shuffled_df, shuffled_phenotype = data_shuffling(df, phenotype, subject_list, seed)        
        # train_meta_x, val_meta_x, train_meta_y, val_meta_y, k_sample_x, test_meta_x, k_sample_y, test_meta_y = data_split_scaling(shuffled_df, shuffled_phenotype, k_num, seed)
        train_x, val_x, train_pheno, val_pheno, train_iq, val_iq, k_sample_x, test_x, k_sample_pheno, test_pheno, k_sample_iq, test_iq = data_split_scaling(shuffled_df, shuffled_phenotype, k_num, seed)
        
        
        # train
        train_x = torch.Tensor(train_x).to(device)
        train_pheno = torch.Tensor(train_pheno).to(device)
        # validation
        val_x = torch.Tensor(val_x).to(device)
        val_pheno = torch.Tensor(val_pheno).to(device)
        # k-shot
        k_sample_x = torch.Tensor(k_sample_x).to(device)
        k_sample_pheno = torch.Tensor(k_sample_pheno).to(device)
        # Rest (Test meta set에서 K-shot을 제외한 나머지 samples)
        test_x = torch.Tensor(test_x).to(device)
        test_pheno = torch.Tensor(test_pheno).to(device)
        
        train_dataset = torch.utils.data.TensorDataset(train_x, train_pheno)
        val_dataset = torch.utils.data.TensorDataset(val_x, val_pheno)
        train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
        val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
        
        k_dataset = torch.utils.data.TensorDataset(k_sample_x, k_sample_pheno)
        test_dataset = torch.utils.data.TensorDataset(test_x, test_pheno)
        k_dataloader = DataLoader(k_dataset, batch_size = k_sample_x.shape[0], shuffle=True, drop_last=True)
        test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)
        


        # 모델 정의 및 Loss function & Optimizer
        model = dnn_4l(train_x.shape[1], 256, 128, 64, 0.2, train_pheno.shape[1]).to(device) 
        model.to(device)
        loss_function = nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        
        # Meta train : Training (100 Epoch)
        for epoch in range(100):  
            model.train()
            for inputs, targets in train_dataloader:
                optimizer.zero_grad()
                outputs = model(inputs) # (batch_size, 58)
                loss = loss_function(outputs, targets)# Fluid Intel을 제외한 나머지 58개의 feature들을 학습할 수 있도록 손실함수 설정 
                loss.backward()
                optimizer.step()  
        
        # validation
        model.eval()
        # Validation prediction
        val_outputs = []
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                outputs = model(inputs)
                val_outputs.append(outputs)
        val_outputs = torch.cat(val_outputs).cpu().detach().numpy()
        # prediction과 phenotype table 모두 transpose를 하기 때문에, (phenotype, samples)가 된다. 
        val_outputs = val_outputs.T
        val_pheno = val_pheno.detach().cpu().numpy().T
        
        # 긱 Phenotype마다, Correlation Coefficient를 저장하기 위한 리스트 
        val_pheno_corrs = []
        for i in range(len(val_outputs)):
            val_pheno_corrs.append(np.corrcoef(val_outputs[i], val_pheno[i])[0][1]) 
            # 일단 Validation에 대한 correlation은 동일한 phenotype의 ground truth와 prediction을 통해서 계산한다. 
            
        print("===============Validation===============")
        print('correlation :',round(sum(val_pheno_corrs)/len(val_pheno_corrs), 4))
        print('max correlation index :', round(val_pheno_corrs.index(max(val_pheno_corrs)),4))
        
        
        # k shot Prediction
        model.eval()
        k_train_outputs = []        
        with torch.no_grad():
            for inputs, targets in k_dataloader:
                K_outputs = model(inputs)
                k_train_outputs.append(K_outputs)
        k_train_outputs = torch.cat(k_train_outputs).cpu().detach().numpy() # k_train_outputs에는 (k_num, 58)의 matrix가 형성? 
        k_sample_pheno = k_sample_pheno.cpu().detach().numpy()
        
        k_shot_age_corrs = []
        k_train_outputs = k_train_outputs.T
        for i in range(len(k_train_outputs)):
            k_shot_age_corrs.append(np.corrcoef(k_train_outputs[i], k_sample_iq)[0][1]) 
        
        print("===============K-shot===============")
        print('k-shot correlation :', round(sum(k_shot_age_corrs)/len(k_shot_age_corrs), 4))
        print('k-shot max correlation index :', round(k_shot_age_corrs.index(max(k_shot_age_corrs)), 4))
            
        # k-shot max correlation index 
        max_corr_pheno_index = k_shot_age_corrs.index(max(k_shot_age_corrs))

        
        # rest Prediction
        test_outputs = [] 
        with torch.no_grad():
            for inputs, targets in test_dataloader:
                outputs = model(inputs)
                test_outputs.append(outputs)
        test_outputs = torch.cat(test_outputs).cpu().detach().numpy()
        test_pheno = test_pheno.cpu().detach().numpy()
        # k shot에서 max correlation index에 해당하는 output만 가져옴
        test_outputs = test_outputs[:, max_corr_pheno_index].reshape(-1, 1)
        
        # age_corr = np.corrcoef(test_outputs[:, 0], test_iq)[0][1]
        # print(f"test_outputs shape : {test_outputs.squeeze().shape}")
        # print(f"test_iq shape : {test_iq.shape}")
        iq_corr = np.corrcoef(test_outputs.squeeze(), test_iq)[0][1] # 왜 슬라이싱 및 인덱싱을 해주어야 하지? -> (141, 1)과 (141,)의 차이
        print("===============Test===============")
        print('IQ corr :', round(iq_corr, 4))
        print('\n')
        corres.append(iq_corr)
        
    if iteration > 1 : 
        print("Mean Corr : ", round(np.mean(corres), 4))
        print("Correlation STD: ", round(np.std(corres),4))
        

### Advanced Stacking

In [147]:
def get_top_k_values_with_indices(k_shot_age_corrs, k_num):
    indexed_lst = list(enumerate(k_shot_age_corrs))  # 리스트의 값과 해당 값의 인덱스를 함께 저장한 리스트 생성
    sorted_lst = sorted(indexed_lst, key=lambda x: x[1], reverse=True)  # 값에 대해 내림차순으로 정렬
    top_k_values = sorted_lst[:k_num]  # 상위 K개의 값만 선택
    top_k_indices = [x[0] for x in top_k_values]  # 선택된 값들의 인덱스 추출
    return top_k_indices

In [148]:
def advanced_stacking(df, phenotype, k_num, iteration = 10):
    
    corres = []
    
    # 반복 실험 
    for seed in range(1, iteration+1):
        
        # Random Seed 설정
        print('iter ', seed)
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)        
        # CUDA를 사용하는 경우 => 이거 반복적으로 동일한 결과가 나오는 지 확인 
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

        # Anat과 pheno_with_age의 index를 랜덤하게 Shuffling -> Train meta set과 Test meta set으로 분리 
        shuffled_df, shuffled_phenotype = data_shuffling(df, phenotype, subject_list, seed)        
        train_x, val_x, train_pheno, val_pheno, train_iq, val_iq, k_sample_x, test_x, k_sample_pheno, test_pheno, k_sample_iq, test_iq = data_split_scaling(shuffled_df, shuffled_phenotype, k_num, seed)
        
        # train
        train_x = torch.Tensor(train_x).to(device)
        train_pheno = torch.Tensor(train_pheno).to(device)
        # validation
        val_x = torch.Tensor(val_x).to(device)
        val_pheno = torch.Tensor(val_pheno).to(device)
        # k-shot
        k_sample_x = torch.Tensor(k_sample_x).to(device)
        k_sample_pheno = torch.Tensor(k_sample_pheno).to(device)
        # Rest (Test meta set에서 K-shot을 제외한 나머지 samples)
        test_x = torch.Tensor(test_x).to(device)
        test_pheno = torch.Tensor(test_pheno).to(device)
        
        train_dataset = torch.utils.data.TensorDataset(train_x, train_pheno)
        val_dataset = torch.utils.data.TensorDataset(val_x, val_pheno)
        train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
        val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
        
        k_dataset = torch.utils.data.TensorDataset(k_sample_x, k_sample_pheno)
        test_dataset = torch.utils.data.TensorDataset(test_x, test_pheno)
        k_dataloader = DataLoader(k_dataset, batch_size = k_sample_x.shape[0], shuffle=True, drop_last=True)
        test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)
        


        # 모델 정의 및 Loss function & Optimizer
        model = dnn_4l(train_x.shape[1], 256, 128, 64, 0.2, train_pheno.shape[1]).to(device)
        model.to(device)
        loss_function = nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        
        # Meta train : Training (100 Epoch)
        for epoch in range(100):  
            model.train()
            for inputs, targets in train_dataloader:
                optimizer.zero_grad()
                outputs = model(inputs) # (batch_size, 58)
                loss = loss_function(outputs, targets)
                loss.backward()
                optimizer.step()

        
        # validation
        model.eval()
        # Validation prediction
        val_outputs = []
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                outputs = model(inputs)
                val_outputs.append(outputs)
        val_outputs = torch.cat(val_outputs).cpu().detach().numpy()
        output_mat = val_outputs.T
        validation_mat = val_pheno.detach().cpu().numpy().T
        
        # 긱 Phenotype마다, Correlation Coefficient를 저장하기 위한 리스트 
        val_pheno_corrs = []
        for i in range(len(output_mat)):
            val_pheno_corrs.append(np.corrcoef(output_mat[i], validation_mat[i])[0][1]) 
            
        print("===============Validation===============")
        print('correlation :', round(sum(val_pheno_corrs)/len(val_pheno_corrs),4))
        print('max correlation index :', val_pheno_corrs.index(max(val_pheno_corrs)))
        
        
        
        # k shot Prediction
        model.eval()
        k_train_outputs = []        
        with torch.no_grad():
            for inputs, targets in k_dataloader:
                K_outputs = model(inputs)
                k_train_outputs.append(K_outputs)
        k_train_outputs = torch.cat(k_train_outputs).cpu().detach().numpy() # k_train_outputs에는 (k_num, 58)의 matrix가 형성? 
        k_sample_pheno = k_sample_pheno.cpu().detach().numpy()
        k_shot_age_corrs = []
        # k_train_outputs을 transpose를 하기 때문에, (phenotype, samples)가 된다. 
        k_train_outputs = k_train_outputs.T
        
        for i in range(len(k_train_outputs)):
            # k-sample들에 대해서 correlation을 구하는 경우에는 phenotype에 대한 prediction과 iq와 관련하여 correlation을 계산 
            k_shot_age_corrs.append(np.corrcoef(k_train_outputs[i], k_sample_iq)[0][1]) 
            
            
            
        
        # K-shot에서 가장 correlation이 높은 k개 features를 선정  
        top_k_indices = get_top_k_values_with_indices(k_shot_age_corrs, k_num)
        # K개의 Feature들이 KRR을 학습 시키는데에 features로 사용된다. 
        # 그리고 다시 Transpose하여 우리가 일반적으로 아는 Row, Column의 특성을 가진다. 
        top_k_train_outputs = k_train_outputs[top_k_indices].T
        
        # 5-fold cross validation
        # 이 부분에서 원래 lambda hyperparameter를 tuning하는 작업이 들어가야 한다. 
        kf = KFold(n_splits=5, shuffle=True, random_state=seed)  
        # 모델 초기화
        krr_model = KernelRidge()
        param_grid = {'alpha':[0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20]}
        # 교차 검증 수행
        # GridSearchCV와 KFold를 사용하여 모델과 파라미터 그리드 설정
        grid_search = GridSearchCV(krr_model, param_grid, cv=KFold(n_splits=5))
        
        # Grid Search 수행하여 최적의 alpha구함 
        grid_search.fit(top_k_train_outputs, k_sample_iq)
        
        krr_model = KernelRidge(grid_search.best_params_['alpha'])
        # K-sample들을 통해서 학습 
        krr_model.fit(top_k_train_outputs, k_sample_iq)
        
        
        
        
        # test sample들을 DNN을 통해서 일단 예측을 한 이후에 K개의 feature들을 선별 
        test_outputs = [] 
        with torch.no_grad():
            for inputs, targets in test_dataloader:
                outputs = model(inputs)
                test_outputs.append(outputs)
        test_outputs = torch.cat(test_outputs).cpu().detach().numpy()
        
        # k shot에서 top_k_indices에 해당하는 output만 가져옴
        top_k_test_outputs = test_outputs[:, top_k_indices]
        
        
        
        # Prediction
        test_outputs = krr_model.predict(top_k_test_outputs)
        # test_meta_y = test_meta_y.cpu().detach().numpy()
        iq_corr = np.corrcoef(test_outputs, test_iq)[0][1]
        print("===============Test===============")
        print(f'IQ corr : {iq_corr:.4f}')
        corres.append(iq_corr)
        
    if iteration > 1 : 
        print(round(np.mean(corres),4))
        print(round(np.std(corres),4)) 
        
    print("===========================================================\n\n")
    
    

### Advanced Fine-Tuning

In [149]:
def advanced_finetune(df, phenotype, k_num, iteration = 10):
    
    corres = []
    
    # 반복 실험 
    for seed in range(1, iteration+1):
        
        # Random Seed 설정
        print('iter ', seed)
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)        
        # CUDA를 사용하는 경우 => 이거 반복적으로 동일한 결과가 나오는 지 확인 
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        
        # Anat과 pheno_with_age의 index를 랜덤하게 Shuffling -> Train meta set과 Test meta set으로 분리 
        shuffled_df, shuffled_phenotype = data_shuffling(df, phenotype, subject_list, seed)        
        train_x, val_x, train_pheno, val_pheno, train_iq, val_iq, k_sample_x, test_x, k_sample_pheno, test_pheno, k_sample_iq, test_iq = data_split_scaling(shuffled_df, shuffled_phenotype, k_num, seed)
        
        
        k_y = np.concatenate((k_sample_pheno, k_sample_iq[:, np.newaxis]), axis=1)
        ft_train_x, ft_val_x, ft_train_y, ft_val_y = train_test_split(k_sample_x, k_y, test_size=0.2, random_state=seed)
        ft_train_iq = ft_train_y[:, -1]
        ft_val_iq = ft_val_y[:, -1]
        ft_train_pheno = ft_train_y[:, :-1]
        ft_val_pheno = ft_val_y[:, :-1]
        
        
        
        # train
        train_x = torch.Tensor(train_x).to(device)
        train_pheno = torch.Tensor(train_pheno).to(device)
        # validation
        val_x = torch.Tensor(val_x).to(device)
        val_pheno = torch.Tensor(val_pheno).to(device)
        # k-shot
        k_sample_x = torch.Tensor(k_sample_x).to(device)
        k_sample_pheno = torch.Tensor(k_sample_pheno).to(device)
        # Rest (Test meta set에서 K-shot을 제외한 나머지 samples)
        test_x = torch.Tensor(test_x).to(device)
        test_pheno = torch.Tensor(test_pheno).to(device)
        # fine tune train
        ft_train_x = torch.Tensor(ft_train_x).to(device)
        ft_train_pheno = torch.Tensor(ft_train_pheno).to(device)
        # fine tune validation
        ft_val_x = torch.Tensor(ft_val_x).to(device)
        ft_val_pheno = torch.Tensor(ft_val_pheno).to(device)        
        
        
        train_dataset = torch.utils.data.TensorDataset(train_x, train_pheno)
        val_dataset = torch.utils.data.TensorDataset(val_x, val_pheno)
        train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
        val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
        
        k_dataset = torch.utils.data.TensorDataset(k_sample_x, k_sample_pheno)
        test_dataset = torch.utils.data.TensorDataset(test_x, test_pheno)
        k_dataloader = DataLoader(k_dataset, batch_size = k_sample_x.shape[0], shuffle=True, drop_last=True)
        test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)
        
        ft_train_dataset = torch.utils.data.TensorDataset(ft_train_x, ft_train_pheno)
        ft_val_dataset = torch.utils.data.TensorDataset(ft_val_x, ft_val_pheno)
        ft_train_dataloader = DataLoader(ft_train_dataset, batch_size= ft_train_x.shape[0], shuffle=True, drop_last=True)
        ft_val_dataloader = DataLoader(ft_val_dataset, batch_size=16, shuffle=False)

        
        
        # 모델 정의 및 Loss function & Optimizer
        model = dnn_4l(train_x.shape[1], 256, 128, 64, 0.2, train_pheno.shape[1]).to(device)
        model.to(device)
        loss_function = nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        
        # Meta train : Training (100 Epoch)
        for epoch in range(100):  
            model.train()
            for inputs, targets in train_dataloader:
                optimizer.zero_grad()
                outputs = model(inputs) # (batch_size, 58)
                loss = loss_function(outputs, targets)
                loss.backward()
                optimizer.step()
        
        
        # validation
        model.eval()
        # Validation prediction
        val_outputs = []
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                outputs = model(inputs)
                val_outputs.append(outputs)
        val_outputs = torch.cat(val_outputs).cpu().detach().numpy()
        output_mat = val_outputs.T
        validation_mat = val_pheno.detach().cpu().numpy().T
        
        # 긱 Phenotype마다, Correlation Coefficient를 저장하기 위한 리스트 
        val_pheno_corrs = []
        for i in range(len(output_mat)):
            val_pheno_corrs.append(np.corrcoef(output_mat[i], validation_mat[i])[0][1]) 
            
        print("===============Validation===============")
        print('correlation :',round(sum(val_pheno_corrs)/len(val_pheno_corrs),4))
        print('max correlation index :', val_pheno_corrs.index(max(val_pheno_corrs)))
        
        
        
        # k shot Prediction을 통해서 DNN에서 IQ를 가장 잘 예측하는 Node를 선정한다. 
        model.eval()
        k_train_outputs = []        
        with torch.no_grad():
            for inputs, targets in k_dataloader:
                K_outputs = model(inputs)
                k_train_outputs.append(K_outputs)
        k_train_outputs = torch.cat(k_train_outputs).cpu().detach().numpy() # k_train_outputs에는 (k_num, 58)의 matrix가 형성? 
        k_shot_age_corrs = []
        k_train_outputs = k_train_outputs.T
        for i in range(len(k_train_outputs)):
            k_shot_age_corrs.append(np.corrcoef(k_train_outputs[i], k_sample_iq)[0][1]) 
        
        print("===============K-shot===============")
        print('k-shot correlation :', round(sum(k_shot_age_corrs)/len(k_shot_age_corrs),4))
        print('k-shot max correlation index :', k_shot_age_corrs.index(max(k_shot_age_corrs)))
            
            
        # k-shot max correlation index 
        max_corr_pheno_index = k_shot_age_corrs.index(max(k_shot_age_corrs))
        print('max corr pheno index', max_corr_pheno_index)
        
        
        # k shot Fine Tuning train
        
        # last 2 layers update
        # 여기에서 가장 마지막 2개의 layer만 학습하도록
        optimizer = torch.optim.SGD([{'params': model.fc3.parameters()}, {'params': model.fc4.parameters()}],
                                    lr = 0.01, momentum = 0.9)
        for epoch in range(100):  
            model.train()
            for inputs, targets in ft_train_dataloader:
                optimizer.zero_grad()
                outputs = model(inputs) # (batch_size, 58)
                loss = loss_function(outputs, targets)
                loss.backward()
                optimizer.step()
                    
        # k shot Fine Tuning validation
        model.eval()
        
        val_outputs = []
        with torch.no_grad():
            for inputs, targets in ft_val_dataloader:
                outputs = model(inputs)
                val_outputs.append(outputs)
        val_outputs = torch.cat(val_outputs).cpu().detach().numpy()
        val_outputs = val_outputs[:, max_corr_pheno_index].reshape(-1, 1)
        iq_corr = np.corrcoef(val_outputs.squeeze(), ft_val_iq)[0][1]
        print("===============K-shot fine tuning validation===============")        
        print('IQ corr', round(iq_corr,4))


        # rest Prediction
   
        test_outputs = [] 
        with torch.no_grad():
            for inputs, targets in test_dataloader:
                outputs = model(inputs)
                test_outputs.append(outputs)
        test_outputs = torch.cat(test_outputs).cpu().detach().numpy()
        # k shot에서 max correlation index에 해당하는 output만 가져옴
        test_outputs = test_outputs[:, max_corr_pheno_index].reshape(-1, 1)
        iq_corr = np.corrcoef(test_outputs.squeeze(), test_iq)[0][1]
        print("===============Test===============")
        print('IQ corr :', round(iq_corr,4))
        corres.append(iq_corr)            
        
    if iteration > 1 : 
        print(round(np.mean(corres),2))
        print(round(np.std(corres),2)) 
        
    print("===========================================================\n\n")

# Experiments

### Basic KRR

**sMRI**

In [82]:
basic_krr(cort_thick, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(cort_thick, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(cort_thick, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0289
final correlation std : 0.11
final correlation list : [-0.2528, 0.0367, -0.0146, 0.1899, 0.0758, 0.0539, 0.1255, 0.0066, 0.0682, -0.0005]



Experiment : k=30, iteration=10
final correlation mean : -0.0326
final correlation std : 0.07
final correlation list : [-0.1303, -0.0769, -0.0075, -0.0443, 0.1472, -0.0649, -0.0567, 0.0105, -0.0762, -0.0264]



Experiment : k=10, iteration=10
final correlation mean : -0.0445
final correlation std : 0.08
final correlation list : [0.1299, -0.1049, -0.0975, -0.1267, -0.1178, 0.0047, 0.0689, -0.093, -0.0946, -0.014]





In [40]:
basic_krr(cort_area, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(cort_area, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(cort_area, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0258
final correlation std : 0.09
final correlation list : [0.0391, 0.129, -0.0684, -0.0245, 0.0684, -0.1924, -0.0099, -0.0753, -0.1322, 0.0084]



Experiment : k=30, iteration=10
final correlation mean : 0.0231
final correlation std : 0.07
final correlation list : [0.0243, -0.0939, 0.0296, 0.1366, -0.0662, 0.0846, -0.0178, 0.0031, -0.009, 0.1397]



Experiment : k=10, iteration=10
final correlation mean : 0.0228
final correlation std : 0.07
final correlation list : [0.0496, -0.1148, 0.1109, 0.0305, 0.1216, 0.0117, -0.0501, -0.03, 0.031, 0.0671]





In [41]:
basic_krr(cort_vol, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(cort_vol, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(cort_vol, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0133
final correlation std : 0.07
final correlation list : [-0.0466, 0.0128, 0.1419, -0.1076, 0.0004, -0.0574, -0.0056, 0.0312, -0.1034, 0.0013]



Experiment : k=30, iteration=10
final correlation mean : -0.0226
final correlation std : 0.08
final correlation list : [0.0073, 0.0117, -0.0156, 0.043, 0.0644, -0.1702, -0.0347, -0.0293, 0.0555, -0.1581]



Experiment : k=10, iteration=10
final correlation mean : 0.018
final correlation std : 0.07
final correlation list : [0.019, -0.0039, -0.0349, 0.0472, -0.017, 0.107, 0.0959, 0.0633, 0.0306, -0.1268]





**TBSS**

In [42]:
basic_krr(tbss_ad, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tbss_ad, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tbss_ad, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.027
final correlation std : 0.09
final correlation list : [0.0944, -0.1199, 0.1212, 0.2, 0.0117, -0.01, -0.0631, -0.0021, -0.0269, 0.0643]



Experiment : k=30, iteration=10
final correlation mean : -0.0579
final correlation std : 0.09
final correlation list : [-0.0833, -0.0202, -0.0448, -0.2008, -0.045, -0.0381, -0.0079, -0.07, -0.1977, 0.1292]



Experiment : k=10, iteration=10
final correlation mean : -0.0041
final correlation std : 0.09
final correlation list : [-0.0192, 0.1962, -0.069, -0.1082, -0.0255, 0.0241, -0.1325, -0.0189, 0.0832, 0.0288]





In [43]:
basic_krr(tbss_fa, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tbss_fa, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tbss_fa, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0101
final correlation std : 0.11
final correlation list : [-0.0309, -0.0359, 0.2022, -0.0574, 0.0531, -0.0634, -0.0514, -0.0943, 0.2182, -0.0388]



Experiment : k=30, iteration=10
final correlation mean : 0.0624
final correlation std : 0.09
final correlation list : [0.1125, 0.1522, -0.0968, 0.0364, 0.1522, 0.1692, 0.097, 0.0556, 0.0025, -0.0565]



Experiment : k=10, iteration=10
final correlation mean : -0.0076
final correlation std : 0.09
final correlation list : [-0.0505, 0.1059, 0.0602, -0.086, 0.0882, 0.054, 0.0553, -0.1368, -0.0456, -0.1208]





In [44]:
basic_krr(tbss_icvf, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tbss_icvf, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tbss_icvf, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0193
final correlation std : 0.05
final correlation list : [-0.0954, -0.0056, -0.0154, 0.0594, -0.0304, -0.0166, -0.038, 0.043, -0.0091, -0.0848]



Experiment : k=30, iteration=10
final correlation mean : 0.0304
final correlation std : 0.08
final correlation list : [0.0062, 0.059, 0.2348, 0.0708, -0.0076, 0.0281, 0.0046, -0.0073, -0.068, -0.0162]



Experiment : k=10, iteration=10
final correlation mean : 0.0119
final correlation std : 0.06
final correlation list : [-0.0004, 0.0256, -0.0057, -0.012, 0.0272, -0.0216, 0.1429, 0.0663, 0.0159, -0.1189]





In [45]:
basic_krr(tbss_isovf, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tbss_isovf, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tbss_isovf, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0216
final correlation std : 0.15
final correlation list : [-0.0521, 0.3069, -0.1204, 0.0546, -0.1025, -0.0392, -0.2146, -0.1455, -0.0457, 0.1429]



Experiment : k=30, iteration=10
final correlation mean : 0.0168
final correlation std : 0.09
final correlation list : [-0.0262, -0.0647, 0.0567, 0.0153, -0.0392, 0.0872, -0.0509, -0.1139, 0.1248, 0.1789]



Experiment : k=10, iteration=10
final correlation mean : -0.0472
final correlation std : 0.09
final correlation list : [-0.1454, -0.0131, 0.11, -0.1526, 0.0308, -0.0107, 0.0299, -0.1636, -0.112, -0.0448]





In [62]:
basic_krr(tbss_md, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tbss_md, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tbss_md, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0162
final correlation std : 0.16
final correlation list : [-0.0367, -0.0528, 0.0795, 0.0019, 0.0416, -0.0903, -0.1843, 0.1717, 0.3756, -0.1441]



Experiment : k=30, iteration=10
final correlation mean : 0.0068
final correlation std : 0.09
final correlation list : [-0.1482, 0.0721, 0.1025, 0.046, -0.0165, 0.0788, 0.0309, -0.0865, 0.1048, -0.1154]



Experiment : k=10, iteration=10
final correlation mean : -0.0033
final correlation std : 0.08
final correlation list : [0.037, 0.1698, -0.0068, -0.1124, 0.0085, 0.0422, -0.0919, -0.0572, -0.0704, 0.0478]





In [108]:
basic_krr(tbss_od, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tbss_od, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tbss_od, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0052
final correlation std : 0.09
final correlation list : [0.138, -0.1046, 0.0859, -0.0376, 0.05, 0.0606, -0.0231, -0.0588, -0.1616, 0.1031]



Experiment : k=30, iteration=10
final correlation mean : -0.0109
final correlation std : 0.11
final correlation list : [0.1428, -0.1679, -0.1662, 0.091, 0.0093, 0.1076, -0.1544, 0.0167, 0.0234, -0.0109]



Experiment : k=10, iteration=10
final correlation mean : -0.0131
final correlation std : 0.06
final correlation list : [0.0124, 0.0788, -0.0611, -0.06, -0.0155, -0.0315, 0.0884, -0.0092, -0.0186, -0.1149]





In [150]:
basic_krr(tbss_rd, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tbss_rd, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tbss_rd, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0275
final correlation std : 0.11
final correlation list : [0.0173, 0.034, -0.1522, -0.1739, -0.2175, 0.1441, 0.0145, -0.0215, -0.0078, 0.0876]



Experiment : k=30, iteration=10
final correlation mean : -0.0481
final correlation std : 0.1
final correlation list : [-0.2279, -0.1064, -0.1298, 0.003, 0.0248, 0.1441, 0.0205, -0.1279, -0.0873, 0.0062]



Experiment : k=10, iteration=10
final correlation mean : 0.0319
final correlation std : 0.09
final correlation list : [0.0186, 0.0018, 0.0045, 0.1631, 0.0025, -0.1698, 0.0154, 0.1251, 0.1259, 0.0322]





**Tractography**

In [151]:
basic_krr(tractography_ad, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tractography_ad, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tractography_ad, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0092
final correlation std : 0.09
final correlation list : [-0.005, -0.034, 0.0311, -0.0088, 0.0904, -0.0931, 0.1003, -0.1644, 0.173, 0.0022]



Experiment : k=30, iteration=10
final correlation mean : 0.0445
final correlation std : 0.09
final correlation list : [0.02, -0.06, 0.06, 0.0746, 0.1646, -0.0783, -0.0767, 0.2161, 0.0484, 0.0761]



Experiment : k=10, iteration=10
final correlation mean : -0.0384
final correlation std : 0.09
final correlation list : [-0.1667, -0.0506, 0.149, 0.0485, -0.096, -0.069, -0.1752, -0.021, 0.0164, -0.0199]





In [152]:
basic_krr(tractography_fa, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tractography_fa, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tractography_fa, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0577
final correlation std : 0.07
final correlation list : [-0.0857, -0.0746, -0.1249, -0.072, 0.0514, 0.0067, -0.0508, 0.0371, -0.1799, -0.0842]



Experiment : k=30, iteration=10
final correlation mean : -0.0006
final correlation std : 0.11
final correlation list : [-0.1384, 0.0981, -0.0339, -0.1282, 0.0622, 0.0688, -0.1246, 0.0319, 0.2259, -0.0675]



Experiment : k=10, iteration=10
final correlation mean : -0.0118
final correlation std : 0.07
final correlation list : [0.0527, -0.0905, 0.0264, 0.0331, 0.0469, -0.1029, 0.0007, -0.1117, -0.0455, 0.073]





In [153]:
basic_krr(tractography_icvf, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tractography_icvf, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tractography_icvf, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0066
final correlation std : 0.09
final correlation list : [-0.0401, -0.0859, -0.0044, -0.1432, 0.0477, 0.032, 0.1213, 0.1194, -0.0777, 0.0967]



Experiment : k=30, iteration=10
final correlation mean : 0.0248
final correlation std : 0.09
final correlation list : [0.1912, 0.1097, -0.0541, 0.0707, -0.0456, -0.0164, -0.126, 0.1015, -0.0538, 0.071]



Experiment : k=10, iteration=10
final correlation mean : 0.0007
final correlation std : 0.07
final correlation list : [-0.0868, 0.0313, 0.0272, 0.0571, 0.0701, 0.042, -0.1337, 0.1065, -0.0452, -0.0616]





In [154]:
basic_krr(tractography_isovf, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tractography_isovf, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tractography_isovf, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.029
final correlation std : 0.07
final correlation list : [0.017, -0.1148, 0.0572, 0.074, 0.0798, -0.0769, 0.1361, 0.0344, 0.0783, 0.0052]



Experiment : k=30, iteration=10
final correlation mean : 0.0138
final correlation std : 0.08
final correlation list : [-0.119, 0.0305, 0.1257, 0.175, -0.0294, 0.0826, -0.0297, -0.05, -0.0349, -0.0124]



Experiment : k=10, iteration=10
final correlation mean : -0.0082
final correlation std : 0.06
final correlation list : [-0.0527, -0.0567, -0.0671, -0.0748, 0.094, 0.0878, 0.0208, 0.029, -0.0061, -0.0566]





In [155]:
basic_krr(tractography_md, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tractography_md, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tractography_md, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0328
final correlation std : 0.1
final correlation list : [-0.1628, -0.1349, 0.0161, -0.073, 0.183, -0.1054, 0.0798, -0.0705, -0.003, -0.0572]



Experiment : k=30, iteration=10
final correlation mean : 0.0027
final correlation std : 0.06
final correlation list : [-0.0973, 0.058, 0.0029, -0.0513, 0.0698, -0.0455, 0.0114, 0.0622, 0.0587, -0.0418]



Experiment : k=10, iteration=10
final correlation mean : -0.0335
final correlation std : 0.1
final correlation list : [-0.1136, 0.0356, 0.1212, -0.0474, -0.0431, -0.2499, -0.0827, -0.0308, -0.0249, 0.1004]





In [156]:
basic_krr(tractography_od, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tractography_od, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tractography_od, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0262
final correlation std : 0.08
final correlation list : [0.0814, 0.123, -0.0323, 0.0704, 0.0209, -0.1778, 0.0875, 0.018, 0.0449, 0.0264]



Experiment : k=30, iteration=10
final correlation mean : -0.0095
final correlation std : 0.09
final correlation list : [0.1146, -0.0194, 0.0868, -0.1322, 0.0865, -0.0467, -0.1245, -0.1176, 0.0044, 0.0528]



Experiment : k=10, iteration=10
final correlation mean : -0.0079
final correlation std : 0.11
final correlation list : [-0.069, -0.1628, -0.0131, 0.0006, 0.1006, 0.0662, -0.0968, 0.0277, -0.1311, 0.1987]





In [157]:
basic_krr(tractography_rd, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tractography_rd, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tractography_rd, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0475
final correlation std : 0.08
final correlation list : [0.1154, 0.1982, -0.1295, 0.0109, 0.0621, -0.0021, 0.1011, 0.0155, 0.0213, 0.0824]



Experiment : k=30, iteration=10
final correlation mean : 0.0062
final correlation std : 0.1
final correlation list : [-0.0179, 0.135, -0.1403, -0.0242, 0.0327, -0.1143, 0.0447, 0.1705, 0.0644, -0.0885]



Experiment : k=10, iteration=10
final correlation mean : -0.0268
final correlation std : 0.05
final correlation list : [-0.0534, 0.0349, -0.1286, 0.0206, -0.0084, 0.021, -0.0195, -0.0447, -0.1004, 0.0104]





In [158]:
basic_krr(tractography_fss, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tractography_fss, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tractography_fss, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0099
final correlation std : 0.11
final correlation list : [-0.0745, 0.0913, -0.0539, 0.1012, -0.0235, -0.0541, -0.1385, 0.1733, -0.1023, 0.1796]



Experiment : k=30, iteration=10
final correlation mean : 0.0028
final correlation std : 0.1
final correlation list : [-0.027, 0.098, 0.1571, -0.0445, -0.1172, 0.0747, 0.0367, 0.0416, -0.1775, -0.0142]



Experiment : k=10, iteration=10
final correlation mean : 0.0007
final correlation std : 0.06
final correlation list : [0.0145, 0.0392, 0.0543, -0.1096, 0.1044, 0.0489, -0.0334, -0.026, -0.0203, -0.0653]





In [159]:
basic_krr(tractography_fssl, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(tractography_fssl, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(tractography_fssl, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0167
final correlation std : 0.08
final correlation list : [-0.1005, -0.0579, 0.0961, -0.044, -0.0216, 0.0825, 0.0872, -0.0643, -0.0107, -0.1341]



Experiment : k=30, iteration=10
final correlation mean : 0.0025
final correlation std : 0.09
final correlation list : [0.009, -0.0583, 0.0802, -0.0349, 0.0996, 0.0677, -0.0045, -0.2061, 0.0866, -0.0139]



Experiment : k=10, iteration=10
final correlation mean : -0.0512
final correlation std : 0.11
final correlation list : [-0.0223, -0.1805, 0.1279, -0.0995, -0.243, 0.0526, -0.0834, -0.1385, 0.0556, 0.0192]





**fMRI**

In [160]:
basic_krr(f_gamb, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(f_gamb, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(f_gamb, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0143
final correlation std : 0.05
final correlation list : [-0.0062, 0.0833, -0.0836, -0.0788, -0.0326, 0.0397, -0.0203, 0.0348, -0.0041, -0.075]



Experiment : k=30, iteration=10
final correlation mean : -0.0336
final correlation std : 0.13
final correlation list : [-0.1949, 0.0807, 0.0601, -0.2236, -0.0247, 0.1163, 0.142, 0.0026, -0.1157, -0.1788]



Experiment : k=10, iteration=10
final correlation mean : 0.0471
final correlation std : 0.09
final correlation list : [0.0653, 0.1726, -0.1135, 0.198, 0.0856, -0.0589, 0.0433, 0.0682, -0.0418, 0.0523]





In [161]:
basic_krr(f_lang, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(f_lang, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(f_lang, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0278
final correlation std : 0.06
final correlation list : [-0.0449, 0.0536, 0.0747, -0.0403, 0.075, 0.0755, 0.1038, 0.0629, -0.0351, -0.0468]



Experiment : k=30, iteration=10
final correlation mean : 0.066
final correlation std : 0.11
final correlation list : [0.1224, -0.0543, 0.0169, -0.0304, 0.177, 0.1002, 0.3026, 0.0248, 0.0668, -0.0662]



Experiment : k=10, iteration=10
final correlation mean : 0.0011
final correlation std : 0.08
final correlation list : [-0.0904, -0.1118, 0.0061, 0.0237, -0.0467, -0.0125, 0.1782, 0.0968, 0.0043, -0.0371]





In [162]:
basic_krr(f_motor, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(f_motor, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(f_motor, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0331
final correlation std : 0.13
final correlation list : [0.1209, -0.0675, -0.012, 0.1675, -0.2467, 0.1624, 0.0901, -0.1028, 0.1877, 0.0313]



Experiment : k=30, iteration=10
final correlation mean : -0.0144
final correlation std : 0.1
final correlation list : [-0.0391, 0.0951, 0.1533, 0.0457, 0.0012, -0.0607, -0.0825, -0.2339, -0.0353, 0.0126]



Experiment : k=10, iteration=10
final correlation mean : 0.021
final correlation std : 0.05
final correlation list : [0.0235, 0.0476, -0.0049, 0.0462, 0.0616, -0.0624, 0.007, 0.1212, -0.0145, -0.0153]





In [163]:
basic_krr(f_rs, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(f_rs, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(f_rs, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0061
final correlation std : 0.07
final correlation list : [0.1095, -0.018, 0.0698, 0.0189, -0.0045, 0.004, 0.0218, -0.0541, -0.0912, -0.1172]



Experiment : k=30, iteration=10
final correlation mean : -0.0011
final correlation std : 0.09
final correlation list : [-0.0782, -0.0052, -0.0371, 0.103, -0.0496, -0.1719, 0.0253, -0.035, 0.1379, 0.0999]



Experiment : k=10, iteration=10
final correlation mean : 0.0162
final correlation std : 0.08
final correlation list : [0.1272, 0.0265, -0.0065, -0.0499, -0.1188, -0.09, 0.0725, 0.0544, 0.0947, 0.052]





In [164]:
basic_krr(f_social, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(f_social, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(f_social, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : -0.0297
final correlation std : 0.1
final correlation list : [-0.0283, -0.1587, 0.0656, -0.1327, -0.118, -0.0926, -0.0676, 0.0488, 0.1484, 0.0376]



Experiment : k=30, iteration=10
final correlation mean : 0.0029
final correlation std : 0.05
final correlation list : [0.0498, 0.036, 0.0128, -0.0621, 0.0187, -0.0139, 0.0907, -0.0492, -0.0405, -0.0129]



Experiment : k=10, iteration=10
final correlation mean : 0.022
final correlation std : 0.07
final correlation list : [0.0721, -0.0086, -0.1002, -0.0625, 0.0739, 0.0851, 0.0277, -0.0546, 0.1165, 0.0706]





In [165]:
basic_krr(f_wm, phenotype, subject_list, KernelRidge(), 50, 10)
basic_krr(f_wm, phenotype, subject_list, KernelRidge(), 30, 10)
basic_krr(f_wm, phenotype, subject_list, KernelRidge(), 10, 10)

Experiment : k=50, iteration=10
final correlation mean : 0.0233
final correlation std : 0.05
final correlation list : [0.0164, 0.0027, 0.0748, -0.0249, 0.1134, 0.066, 0.0245, 0.004, -0.0844, 0.0403]



Experiment : k=30, iteration=10
final correlation mean : -0.023
final correlation std : 0.11
final correlation list : [0.03, -0.1855, 0.1274, 0.1632, -0.0623, 0.0624, -0.0826, -0.1516, -0.0563, -0.0744]



Experiment : k=10, iteration=10
final correlation mean : -0.0254
final correlation std : 0.07
final correlation list : [0.03, 0.0627, 0.0681, -0.1549, -0.0759, 0.0405, -0.0389, -0.0548, -0.029, -0.1014]





### Basic DNN 

In [168]:
basic_dnn(cort_thick, phenotype, 10, 10)
basic_dnn(cort_thick, phenotype, 30, 10)
basic_dnn(cort_thick, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.0201
max correlation index : 33
===============K-shot===============
k-shot correlation : -0.0685
k-shot max correlation index : 21
===============Test===============
IQ corr : 0.0461


iter :  2
===============Validation===============
correlation : -0.0225
max correlation index : 16
===============K-shot===============
k-shot correlation : 0.0121
k-shot max correlation index : 46
===============Test===============
IQ corr : -0.0318


iter :  3
===============Validation===============
correlation : -0.017
max correlation index : 53
===============K-shot===============
k-shot correlation : 0.0027
k-shot max correlation index : 55
===============Test===============
IQ corr : 0.1999


iter :  4
===============Validation===============
correlation : -0.0057
max correlation index : 15
===============K-shot===============
k-shot correlation : -0.0253
k-shot max correlation index : 33
===============Test===============
IQ cor

In [169]:
basic_dnn(cort_area, phenotype, 10, 10)
basic_dnn(cort_area, phenotype, 30, 10)
basic_dnn(cort_area, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.0259
max correlation index : 48
===============K-shot===============
k-shot correlation : 0.1429
k-shot max correlation index : 8
===============Test===============
IQ corr : 0.0991


iter :  2
===============Validation===============
correlation : -0.0434
max correlation index : 19
===============K-shot===============
k-shot correlation : -0.0502
k-shot max correlation index : 28
===============Test===============
IQ corr : -0.0986


iter :  3
===============Validation===============
correlation : 0.0241
max correlation index : 46
===============K-shot===============
k-shot correlation : 0.0338
k-shot max correlation index : 29
===============Test===============
IQ corr : -0.0952


iter :  4
===============Validation===============
correlation : 0.0032
max correlation index : 10
===============K-shot===============
k-shot correlation : -0.1123
k-shot max correlation index : 32
===============Test===============
IQ corr

In [170]:
basic_dnn(cort_vol, phenotype, 10, 10)
basic_dnn(cort_vol, phenotype, 30, 10)
basic_dnn(cort_vol, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.0324
max correlation index : 25
===============K-shot===============
k-shot correlation : 0.0486
k-shot max correlation index : 21
===============Test===============
IQ corr : -0.0682


iter :  2
===============Validation===============
correlation : 0.0169
max correlation index : 52
===============K-shot===============
k-shot correlation : 0.0523
k-shot max correlation index : 1
===============Test===============
IQ corr : 0.1246


iter :  3
===============Validation===============
correlation : -0.0205
max correlation index : 44
===============K-shot===============
k-shot correlation : 0.1036
k-shot max correlation index : 57
===============Test===============
IQ corr : -0.0588


iter :  4
===============Validation===============
correlation : 0.0086
max correlation index : 54
===============K-shot===============
k-shot correlation : 0.1226
k-shot max correlation index : 12
===============Test===============
IQ corr :

In [171]:
basic_dnn(tbss_ad, phenotype, 10, 10)
basic_dnn(tbss_ad, phenotype, 30, 10)
basic_dnn(tbss_ad, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.0301
max correlation index : 32
===============K-shot===============
k-shot correlation : 0.1972
k-shot max correlation index : 30
===============Test===============
IQ corr : -0.1172


iter :  2
===============Validation===============
correlation : 0.0245
max correlation index : 50
===============K-shot===============
k-shot correlation : 0.0578
k-shot max correlation index : 43
===============Test===============
IQ corr : -0.0249


iter :  3
===============Validation===============
correlation : -0.0312
max correlation index : 19
===============K-shot===============
k-shot correlation : -0.0793
k-shot max correlation index : 53
===============Test===============
IQ corr : -0.0836


iter :  4
===============Validation===============
correlation : 0.0034
max correlation index : 45
===============K-shot===============
k-shot correlation : -0.1257
k-shot max correlation index : 0
===============Test===============
IQ cor

In [172]:
basic_dnn(tbss_fa, phenotype, 10, 10)
basic_dnn(tbss_fa, phenotype, 30, 10)
basic_dnn(tbss_fa, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0193
max correlation index : 23
===============K-shot===============
k-shot correlation : 0.0184
k-shot max correlation index : 35
===============Test===============
IQ corr : -0.2494


iter :  2
===============Validation===============
correlation : -0.0077
max correlation index : 42
===============K-shot===============
k-shot correlation : 0.0434
k-shot max correlation index : 48
===============Test===============
IQ corr : 0.0486


iter :  3
===============Validation===============
correlation : -0.0192
max correlation index : 4
===============K-shot===============
k-shot correlation : -0.1625
k-shot max correlation index : 20
===============Test===============
IQ corr : 0.1329


iter :  4
===============Validation===============
correlation : 0.0366
max correlation index : 0
===============K-shot===============
k-shot correlation : -0.0492
k-shot max correlation index : 36
===============Test===============
IQ corr

In [173]:
basic_dnn(tbss_icvf, phenotype, 10, 10)
basic_dnn(tbss_icvf, phenotype, 30, 10)
basic_dnn(tbss_icvf, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.0183
max correlation index : 17
===============K-shot===============
k-shot correlation : -0.076
k-shot max correlation index : 39
===============Test===============
IQ corr : -0.098


iter :  2
===============Validation===============
correlation : -0.0285
max correlation index : 27
===============K-shot===============
k-shot correlation : -0.1899
k-shot max correlation index : 28
===============Test===============
IQ corr : -0.0338


iter :  3
===============Validation===============
correlation : -0.0553
max correlation index : 6
===============K-shot===============
k-shot correlation : -0.0893
k-shot max correlation index : 21
===============Test===============
IQ corr : 0.0342


iter :  4
===============Validation===============
correlation : -0.0068
max correlation index : 40
===============K-shot===============
k-shot correlation : 0.049
k-shot max correlation index : 32
===============Test===============
IQ corr

In [174]:
basic_dnn(tbss_isovf, phenotype, 10, 10)
basic_dnn(tbss_isovf, phenotype, 30, 10)
basic_dnn(tbss_isovf, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0284
max correlation index : 16
===============K-shot===============
k-shot correlation : 0.0829
k-shot max correlation index : 9
===============Test===============
IQ corr : 0.0297


iter :  2
===============Validation===============
correlation : -0.0332
max correlation index : 43
===============K-shot===============
k-shot correlation : -0.0381
k-shot max correlation index : 51
===============Test===============
IQ corr : -0.1415


iter :  3
===============Validation===============
correlation : 0.006
max correlation index : 37
===============K-shot===============
k-shot correlation : -0.0973
k-shot max correlation index : 0
===============Test===============
IQ corr : -0.0861


iter :  4
===============Validation===============
correlation : -0.046
max correlation index : 45
===============K-shot===============
k-shot correlation : -0.0245
k-shot max correlation index : 28
===============Test===============
IQ corr

In [175]:
basic_dnn(tbss_md, phenotype, 10, 10)
basic_dnn(tbss_md, phenotype, 30, 10)
basic_dnn(tbss_md, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.01
max correlation index : 21
===============K-shot===============
k-shot correlation : -0.0079
k-shot max correlation index : 50
===============Test===============
IQ corr : -0.063


iter :  2
===============Validation===============
correlation : -0.0105
max correlation index : 12
===============K-shot===============
k-shot correlation : -0.0337
k-shot max correlation index : 40
===============Test===============
IQ corr : -0.0228


iter :  3
===============Validation===============
correlation : 0.0194
max correlation index : 49
===============K-shot===============
k-shot correlation : 0.0504
k-shot max correlation index : 35
===============Test===============
IQ corr : 0.0093


iter :  4
===============Validation===============
correlation : 0.0366
max correlation index : 11
===============K-shot===============
k-shot correlation : -0.0542
k-shot max correlation index : 0
===============Test===============
IQ corr 

In [176]:
basic_dnn(tbss_od, phenotype, 10, 10)
basic_dnn(tbss_od, phenotype, 30, 10)
basic_dnn(tbss_od, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.002
max correlation index : 45
===============K-shot===============
k-shot correlation : -0.155
k-shot max correlation index : 32
===============Test===============
IQ corr : 0.1299


iter :  2
===============Validation===============
correlation : 0.0089
max correlation index : 28
===============K-shot===============
k-shot correlation : 0.0745
k-shot max correlation index : 52
===============Test===============
IQ corr : -0.0637


iter :  3
===============Validation===============
correlation : -0.0224
max correlation index : 6
===============K-shot===============
k-shot correlation : 0.0256
k-shot max correlation index : 13
===============Test===============
IQ corr : -0.1194


iter :  4
===============Validation===============
correlation : -0.0237
max correlation index : 10
===============K-shot===============
k-shot correlation : -0.2202
k-shot max correlation index : 30
===============Test===============
IQ corr

In [177]:
basic_dnn(tbss_rd, phenotype, 10, 10)
basic_dnn(tbss_rd, phenotype, 30, 10)
basic_dnn(tbss_rd, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.0006
max correlation index : 20
===============K-shot===============
k-shot correlation : 0.0589
k-shot max correlation index : 1
===============Test===============
IQ corr : -0.0235


iter :  2
===============Validation===============
correlation : -0.0091
max correlation index : 39
===============K-shot===============
k-shot correlation : 0.0948
k-shot max correlation index : 9
===============Test===============
IQ corr : -0.0212


iter :  3
===============Validation===============
correlation : 0.0415
max correlation index : 15
===============K-shot===============
k-shot correlation : -0.0211
k-shot max correlation index : 27
===============Test===============
IQ corr : -0.0485


iter :  4
===============Validation===============
correlation : 0.0087
max correlation index : 37
===============K-shot===============
k-shot correlation : -0.1388
k-shot max correlation index : 18
===============Test===============
IQ corr

In [178]:
basic_dnn(tractography_ad, phenotype, 10, 10)
basic_dnn(tractography_ad, phenotype, 30, 10)
basic_dnn(tractography_ad, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.0062
max correlation index : 14
===============K-shot===============
k-shot correlation : -0.0942
k-shot max correlation index : 50
===============Test===============
IQ corr : 0.1686


iter :  2
===============Validation===============
correlation : -0.0284
max correlation index : 18
===============K-shot===============
k-shot correlation : -0.1979
k-shot max correlation index : 16
===============Test===============
IQ corr : -0.0602


iter :  3
===============Validation===============
correlation : -0.0166
max correlation index : 54
===============K-shot===============
k-shot correlation : 0.0063
k-shot max correlation index : 33
===============Test===============
IQ corr : -0.0285


iter :  4
===============Validation===============
correlation : 0.0266
max correlation index : 14
===============K-shot===============
k-shot correlation : -0.0549
k-shot max correlation index : 43
===============Test===============
IQ c

In [179]:
basic_dnn(tractography_fa, phenotype, 10, 10)
basic_dnn(tractography_fa, phenotype, 30, 10)
basic_dnn(tractography_fa, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0203
max correlation index : 6
===============K-shot===============
k-shot correlation : -0.0925
k-shot max correlation index : 0
===============Test===============
IQ corr : 0.1162


iter :  2
===============Validation===============
correlation : 0.0527
max correlation index : 51
===============K-shot===============
k-shot correlation : 0.0101
k-shot max correlation index : 35
===============Test===============
IQ corr : 0.1038


iter :  3
===============Validation===============
correlation : 0.0309
max correlation index : 30
===============K-shot===============
k-shot correlation : 0.0612
k-shot max correlation index : 8
===============Test===============
IQ corr : -0.1431


iter :  4
===============Validation===============
correlation : 0.02
max correlation index : 25
===============K-shot===============
k-shot correlation : 0.1141
k-shot max correlation index : 37
===============Test===============
IQ corr : -0.

In [180]:
basic_dnn(tractography_icvf, phenotype, 10, 10)
basic_dnn(tractography_icvf, phenotype, 30, 10)
basic_dnn(tractography_icvf, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0237
max correlation index : 3
===============K-shot===============
k-shot correlation : -0.0423
k-shot max correlation index : 32
===============Test===============
IQ corr : -0.0505


iter :  2
===============Validation===============
correlation : 0.0106
max correlation index : 3
===============K-shot===============
k-shot correlation : 0.3073
k-shot max correlation index : 10
===============Test===============
IQ corr : -0.0455


iter :  3
===============Validation===============
correlation : -0.0417
max correlation index : 9
===============K-shot===============
k-shot correlation : -0.0907
k-shot max correlation index : 0
===============Test===============
IQ corr : -0.1244


iter :  4
===============Validation===============
correlation : -0.0404
max correlation index : 12
===============K-shot===============
k-shot correlation : 0.0229
k-shot max correlation index : 25
===============Test===============
IQ corr

In [181]:
basic_dnn(tractography_isovf, phenotype, 10, 10)
basic_dnn(tractography_isovf, phenotype, 30, 10)
basic_dnn(tractography_isovf, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0325
max correlation index : 16
===============K-shot===============
k-shot correlation : -0.0795
k-shot max correlation index : 32
===============Test===============
IQ corr : -0.0407


iter :  2
===============Validation===============
correlation : 0.0082
max correlation index : 29
===============K-shot===============
k-shot correlation : 0.0577
k-shot max correlation index : 57
===============Test===============
IQ corr : -0.0091


iter :  3
===============Validation===============
correlation : 0.0141
max correlation index : 17
===============K-shot===============
k-shot correlation : -0.0243
k-shot max correlation index : 15
===============Test===============
IQ corr : -0.0219


iter :  4
===============Validation===============
correlation : -0.0097
max correlation index : 37
===============K-shot===============
k-shot correlation : -0.1004
k-shot max correlation index : 53
===============Test===============
IQ 

In [182]:
basic_dnn(tractography_md, phenotype, 10, 10)
basic_dnn(tractography_md, phenotype, 30, 10)
basic_dnn(tractography_md, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0203
max correlation index : 37
===============K-shot===============
k-shot correlation : 0.0339
k-shot max correlation index : 44
===============Test===============
IQ corr : -0.0327


iter :  2
===============Validation===============
correlation : 0.0021
max correlation index : 38
===============K-shot===============
k-shot correlation : -0.0971
k-shot max correlation index : 20
===============Test===============
IQ corr : 0.1367


iter :  3
===============Validation===============
correlation : -0.0085
max correlation index : 49
===============K-shot===============
k-shot correlation : -0.0608
k-shot max correlation index : 55
===============Test===============
IQ corr : -0.107


iter :  4
===============Validation===============
correlation : -0.0208
max correlation index : 40
===============K-shot===============
k-shot correlation : -0.0341
k-shot max correlation index : 32
===============Test===============
IQ c

In [183]:
basic_dnn(tractography_od, phenotype, 10, 10)
basic_dnn(tractography_od, phenotype, 30, 10)
basic_dnn(tractography_od, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0152
max correlation index : 16
===============K-shot===============
k-shot correlation : 0.1712
k-shot max correlation index : 8
===============Test===============
IQ corr : -0.0204


iter :  2
===============Validation===============
correlation : 0.0035
max correlation index : 7
===============K-shot===============
k-shot correlation : -0.0184
k-shot max correlation index : 42
===============Test===============
IQ corr : 0.1183


iter :  3
===============Validation===============
correlation : 0.0265
max correlation index : 18
===============K-shot===============
k-shot correlation : -0.0075
k-shot max correlation index : 49
===============Test===============
IQ corr : -0.0499


iter :  4
===============Validation===============
correlation : -0.0049
max correlation index : 40
===============K-shot===============
k-shot correlation : 0.0383
k-shot max correlation index : 1
===============Test===============
IQ corr 

In [184]:
basic_dnn(tractography_rd, phenotype, 10, 10)
basic_dnn(tractography_rd, phenotype, 30, 10)
basic_dnn(tractography_rd, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0181
max correlation index : 35
===============K-shot===============
k-shot correlation : -0.0932
k-shot max correlation index : 55
===============Test===============
IQ corr : -0.0641


iter :  2
===============Validation===============
correlation : 0.0057
max correlation index : 0
===============K-shot===============
k-shot correlation : 0.059
k-shot max correlation index : 19
===============Test===============
IQ corr : 0.0943


iter :  3
===============Validation===============
correlation : 0.0023
max correlation index : 41
===============K-shot===============
k-shot correlation : 0.1465
k-shot max correlation index : 35
===============Test===============
IQ corr : 0.0181


iter :  4
===============Validation===============
correlation : 0.0144
max correlation index : 28
===============K-shot===============
k-shot correlation : 0.1578
k-shot max correlation index : 5
===============Test===============
IQ corr : 0

In [185]:
basic_dnn(tractography_fss, phenotype, 10, 10)
basic_dnn(tractography_fss, phenotype, 30, 10)
basic_dnn(tractography_fss, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0345
max correlation index : 38
===============K-shot===============
k-shot correlation : -0.1248
k-shot max correlation index : 54
===============Test===============
IQ corr : -0.0169


iter :  2
===============Validation===============
correlation : -0.021
max correlation index : 7
===============K-shot===============
k-shot correlation : -0.046
k-shot max correlation index : 12
===============Test===============
IQ corr : -0.006


iter :  3
===============Validation===============
correlation : 0.0189
max correlation index : 33
===============K-shot===============
k-shot correlation : -0.0552
k-shot max correlation index : 32
===============Test===============
IQ corr : -0.0851


iter :  4
===============Validation===============
correlation : -0.0241
max correlation index : 41
===============K-shot===============
k-shot correlation : 0.0767
k-shot max correlation index : 49
===============Test===============
IQ cor

In [186]:
basic_dnn(tractography_fssl, phenotype, 10, 10)
basic_dnn(tractography_fssl, phenotype, 30, 10)
basic_dnn(tractography_fssl, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0218
max correlation index : 10
===============K-shot===============
k-shot correlation : 0.0356
k-shot max correlation index : 36
===============Test===============
IQ corr : -0.0369


iter :  2
===============Validation===============
correlation : 0.0408
max correlation index : 33
===============K-shot===============
k-shot correlation : 0.1903
k-shot max correlation index : 43
===============Test===============
IQ corr : -0.0394


iter :  3
===============Validation===============
correlation : 0.0102
max correlation index : 26
===============K-shot===============
k-shot correlation : 0.0053
k-shot max correlation index : 54
===============Test===============
IQ corr : -0.0671


iter :  4
===============Validation===============
correlation : -0.0307
max correlation index : 40
===============K-shot===============
k-shot correlation : 0.0549
k-shot max correlation index : 16
===============Test===============
IQ cor

In [187]:
basic_dnn(f_gamb, phenotype, 10, 10)
basic_dnn(f_gamb, phenotype, 30, 10)
basic_dnn(f_gamb, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.0199
max correlation index : 4
===============K-shot===============
k-shot correlation : 0.051
k-shot max correlation index : 16
===============Test===============
IQ corr : -0.138


iter :  2
===============Validation===============
correlation : 0.0341
max correlation index : 45
===============K-shot===============
k-shot correlation : 0.0871
k-shot max correlation index : 16
===============Test===============
IQ corr : -0.0486


iter :  3
===============Validation===============
correlation : -0.022
max correlation index : 14
===============K-shot===============
k-shot correlation : -0.028
k-shot max correlation index : 19
===============Test===============
IQ corr : 0.0519


iter :  4
===============Validation===============
correlation : 0.0111
max correlation index : 1
===============K-shot===============
k-shot correlation : 0.0139
k-shot max correlation index : 27
===============Test===============
IQ corr : 0.1

In [188]:
basic_dnn(f_lang, phenotype, 10, 10)
basic_dnn(f_lang, phenotype, 30, 10)
basic_dnn(f_lang, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.009
max correlation index : 14
===============K-shot===============
k-shot correlation : 0.0659
k-shot max correlation index : 37
===============Test===============
IQ corr : 0.0193


iter :  2
===============Validation===============
correlation : -0.0051
max correlation index : 38
===============K-shot===============
k-shot correlation : -0.1084
k-shot max correlation index : 7
===============Test===============
IQ corr : -0.2003


iter :  3
===============Validation===============
correlation : -0.0121
max correlation index : 43
===============K-shot===============
k-shot correlation : -0.0788
k-shot max correlation index : 21
===============Test===============
IQ corr : 0.0523


iter :  4
===============Validation===============
correlation : 0.0478
max correlation index : 44
===============K-shot===============
k-shot correlation : 0.1862
k-shot max correlation index : 45
===============Test===============
IQ corr 

In [189]:
basic_dnn(f_motor, phenotype, 10, 10)
basic_dnn(f_motor, phenotype, 30, 10)
basic_dnn(f_motor, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.033
max correlation index : 33
===============K-shot===============
k-shot correlation : -0.0714
k-shot max correlation index : 34
===============Test===============
IQ corr : 0.1465


iter :  2
===============Validation===============
correlation : -0.0117
max correlation index : 47
===============K-shot===============
k-shot correlation : 0.0045
k-shot max correlation index : 14
===============Test===============
IQ corr : -0.0225


iter :  3
===============Validation===============
correlation : -0.0023
max correlation index : 8
===============K-shot===============
k-shot correlation : -0.1471
k-shot max correlation index : 53
===============Test===============
IQ corr : 0.067


iter :  4
===============Validation===============
correlation : -0.0135
max correlation index : 52
===============K-shot===============
k-shot correlation : 0.0784
k-shot max correlation index : 9
===============Test===============
IQ corr :

In [190]:
basic_dnn(f_rs, phenotype, 10, 10)
basic_dnn(f_rs, phenotype, 30, 10)
basic_dnn(f_rs, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : -0.0035
max correlation index : 14
===============K-shot===============
k-shot correlation : -0.0892
k-shot max correlation index : 10
===============Test===============
IQ corr : 0.0613


iter :  2
===============Validation===============
correlation : -0.0363
max correlation index : 15
===============K-shot===============
k-shot correlation : -0.0183
k-shot max correlation index : 35
===============Test===============
IQ corr : -0.0005


iter :  3
===============Validation===============
correlation : -0.0065
max correlation index : 21
===============K-shot===============
k-shot correlation : 0.0651
k-shot max correlation index : 30
===============Test===============
IQ corr : -0.0755


iter :  4
===============Validation===============
correlation : -0.035
max correlation index : 39
===============K-shot===============
k-shot correlation : 0.0093
k-shot max correlation index : 28
===============Test===============
IQ c

In [191]:
basic_dnn(f_social, phenotype, 10, 10)
basic_dnn(f_social, phenotype, 30, 10)
basic_dnn(f_social, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.0137
max correlation index : 38
===============K-shot===============
k-shot correlation : -0.0743
k-shot max correlation index : 57
===============Test===============
IQ corr : -0.0517


iter :  2
===============Validation===============
correlation : 0.0393
max correlation index : 14
===============K-shot===============
k-shot correlation : -0.0566
k-shot max correlation index : 13
===============Test===============
IQ corr : 0.1186


iter :  3
===============Validation===============
correlation : -0.0171
max correlation index : 3
===============K-shot===============
k-shot correlation : -0.0904
k-shot max correlation index : 50
===============Test===============
IQ corr : 0.0141


iter :  4
===============Validation===============
correlation : -0.0114
max correlation index : 3
===============K-shot===============
k-shot correlation : 0.0528
k-shot max correlation index : 28
===============Test===============
IQ corr

In [192]:
basic_dnn(f_wm, phenotype, 10, 10)
basic_dnn(f_wm, phenotype, 30, 10)
basic_dnn(f_wm, phenotype, 50, 10)

iter :  1
===============Validation===============
correlation : 0.0301
max correlation index : 43
===============K-shot===============
k-shot correlation : 0.0379
k-shot max correlation index : 38
===============Test===============
IQ corr : -0.0086


iter :  2
===============Validation===============
correlation : 0.0166
max correlation index : 50
===============K-shot===============
k-shot correlation : -0.0036
k-shot max correlation index : 30
===============Test===============
IQ corr : 0.0497


iter :  3
===============Validation===============
correlation : -0.0301
max correlation index : 1
===============K-shot===============
k-shot correlation : 0.0736
k-shot max correlation index : 23
===============Test===============
IQ corr : 0.0662


iter :  4
===============Validation===============
correlation : -0.0563
max correlation index : 18
===============K-shot===============
k-shot correlation : 0.1064
k-shot max correlation index : 10
===============Test===============
IQ corr 

### Advanced Fine-Tuning

**sMRI**

In [193]:
advanced_finetune(cort_thick, phenotype, 50, 10)
advanced_finetune(cort_thick, phenotype, 30, 10)
advanced_finetune(cort_thick, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0466
max correlation index : 14
===============K-shot===============
k-shot correlation : 0.0285
k-shot max correlation index : 4
max corr pheno index 4
===============K-shot fine tuning validation===============
IQ corr -0.6696
===============Test===============
IQ corr : -0.0244
iter  2
===============Validation===============
correlation : -0.0136
max correlation index : 35
===============K-shot===============
k-shot correlation : 0.0375
k-shot max correlation index : 30
max corr pheno index 30
===============K-shot fine tuning validation===============
IQ corr -0.5523
===============Test===============
IQ corr : 0.1148
iter  3
===============Validation===============
correlation : -0.0127
max correlation index : 21
===============K-shot===============
k-shot correlation : -0.0044
k-shot max correlation index : 19
max corr pheno index 19
===============K-shot fine tuning validation===============
IQ corr -0.2558
======

In [194]:
advanced_finetune(cort_area, phenotype, 50, 10)
advanced_finetune(cort_area, phenotype, 30, 10)
advanced_finetune(cort_area, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0039
max correlation index : 38
===============K-shot===============
k-shot correlation : 0.0058
k-shot max correlation index : 57
max corr pheno index 57
===============K-shot fine tuning validation===============
IQ corr -0.3716
===============Test===============
IQ corr : 0.0837
iter  2
===============Validation===============
correlation : 0.011
max correlation index : 29
===============K-shot===============
k-shot correlation : -0.004
k-shot max correlation index : 40
max corr pheno index 40
===============K-shot fine tuning validation===============
IQ corr -0.0435
===============Test===============
IQ corr : -0.1116
iter  3
===============Validation===============
correlation : -0.0042
max correlation index : 0
===============K-shot===============
k-shot correlation : 0.0175
k-shot max correlation index : 26
max corr pheno index 26
===============K-shot fine tuning validation===============
IQ corr 0.1604
========

In [195]:
advanced_finetune(cort_vol, phenotype, 50, 10)
advanced_finetune(cort_vol, phenotype, 30, 10)
advanced_finetune(cort_vol, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0159
max correlation index : 50
===============K-shot===============
k-shot correlation : -0.0024
k-shot max correlation index : 14
max corr pheno index 14
===============K-shot fine tuning validation===============
IQ corr -0.5405
===============Test===============
IQ corr : -0.1835
iter  2
===============Validation===============
correlation : 0.0067
max correlation index : 6
===============K-shot===============
k-shot correlation : 0.0168
k-shot max correlation index : 21
max corr pheno index 21
===============K-shot fine tuning validation===============
IQ corr -0.4827
===============Test===============
IQ corr : 0.0229
iter  3
===============Validation===============
correlation : -0.0034
max correlation index : 37
===============K-shot===============
k-shot correlation : -0.0062
k-shot max correlation index : 49
max corr pheno index 49
===============K-shot fine tuning validation===============
IQ corr 0.481
=======

**TBSS**

In [196]:
advanced_finetune(tbss_ad, phenotype, 50, 10)
advanced_finetune(tbss_ad, phenotype, 30, 10)
advanced_finetune(tbss_ad, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0076
max correlation index : 13
===============K-shot===============
k-shot correlation : 0.0069
k-shot max correlation index : 9
max corr pheno index 9
===============K-shot fine tuning validation===============
IQ corr 0.0942
===============Test===============
IQ corr : -0.0319
iter  2
===============Validation===============
correlation : -0.0249
max correlation index : 37
===============K-shot===============
k-shot correlation : -0.04
k-shot max correlation index : 21
max corr pheno index 21
===============K-shot fine tuning validation===============
IQ corr 0.3599
===============Test===============
IQ corr : -0.0864
iter  3
===============Validation===============
correlation : 0.0112
max correlation index : 32
===============K-shot===============
k-shot correlation : -0.1143
k-shot max correlation index : 0
max corr pheno index 0
===============K-shot fine tuning validation===============
IQ corr -0.454
============

In [197]:
advanced_finetune(tbss_fa, phenotype, 50, 10)
advanced_finetune(tbss_fa, phenotype, 30, 10)
advanced_finetune(tbss_fa, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0067
max correlation index : 12
===============K-shot===============
k-shot correlation : -0.0244
k-shot max correlation index : 20
max corr pheno index 20
===============K-shot fine tuning validation===============
IQ corr 0.1756
===============Test===============
IQ corr : 0.0182
iter  2
===============Validation===============
correlation : -0.0019
max correlation index : 40
===============K-shot===============
k-shot correlation : 0.0066
k-shot max correlation index : 0
max corr pheno index 0
===============K-shot fine tuning validation===============
IQ corr -0.8138
===============Test===============
IQ corr : 0.1558
iter  3
===============Validation===============
correlation : 0.0173
max correlation index : 23
===============K-shot===============
k-shot correlation : -0.0074
k-shot max correlation index : 27
max corr pheno index 27
===============K-shot fine tuning validation===============
IQ corr -0.5481
=======

In [198]:
advanced_finetune(tbss_icvf, phenotype, 50, 10)
advanced_finetune(tbss_icvf, phenotype, 30, 10)
advanced_finetune(tbss_icvf, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.027
max correlation index : 40
===============K-shot===============
k-shot correlation : 0.0863
k-shot max correlation index : 50
max corr pheno index 50
===============K-shot fine tuning validation===============
IQ corr 0.1293
===============Test===============
IQ corr : -0.1228
iter  2
===============Validation===============
correlation : -0.0125
max correlation index : 35
===============K-shot===============
k-shot correlation : 0.0586
k-shot max correlation index : 8
max corr pheno index 8
===============K-shot fine tuning validation===============
IQ corr 0.0074
===============Test===============
IQ corr : -0.1372
iter  3
===============Validation===============
correlation : -0.0338
max correlation index : 16
===============K-shot===============
k-shot correlation : 0.0539
k-shot max correlation index : 16
max corr pheno index 16
===============K-shot fine tuning validation===============
IQ corr -0.5481
=========

In [199]:
advanced_finetune(tbss_isovf, phenotype, 50, 10)
advanced_finetune(tbss_isovf, phenotype, 30, 10)
advanced_finetune(tbss_isovf, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0025
max correlation index : 23
===============K-shot===============
k-shot correlation : 0.012
k-shot max correlation index : 23
max corr pheno index 23
===============K-shot fine tuning validation===============
IQ corr -0.2421
===============Test===============
IQ corr : -0.0115
iter  2
===============Validation===============
correlation : -0.0305
max correlation index : 22
===============K-shot===============
k-shot correlation : 0.0543
k-shot max correlation index : 45
max corr pheno index 45
===============K-shot fine tuning validation===============
IQ corr -0.259
===============Test===============
IQ corr : 0.229
iter  3
===============Validation===============
correlation : -0.0028
max correlation index : 17
===============K-shot===============
k-shot correlation : -0.0403
k-shot max correlation index : 22
max corr pheno index 22
===============K-shot fine tuning validation===============
IQ corr 0.2331
=======

In [200]:
advanced_finetune(tbss_md, phenotype, 50, 10)
advanced_finetune(tbss_md, phenotype, 30, 10)
advanced_finetune(tbss_md, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0259
max correlation index : 45
===============K-shot===============
k-shot correlation : -0.0823
k-shot max correlation index : 21
max corr pheno index 21
===============K-shot fine tuning validation===============
IQ corr -0.5316
===============Test===============
IQ corr : -0.0819
iter  2
===============Validation===============
correlation : -0.0444
max correlation index : 19
===============K-shot===============
k-shot correlation : -0.0331
k-shot max correlation index : 7
max corr pheno index 7
===============K-shot fine tuning validation===============
IQ corr 0.308
===============Test===============
IQ corr : -0.0621
iter  3
===============Validation===============
correlation : -0.0398
max correlation index : 20
===============K-shot===============
k-shot correlation : 0.0009
k-shot max correlation index : 30
max corr pheno index 30
===============K-shot fine tuning validation===============
IQ corr 0.2241
=======

In [201]:
advanced_finetune(tbss_od, phenotype, 50, 10)
advanced_finetune(tbss_od, phenotype, 30, 10)
advanced_finetune(tbss_od, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0073
max correlation index : 52
===============K-shot===============
k-shot correlation : 0.0221
k-shot max correlation index : 0
max corr pheno index 0
===============K-shot fine tuning validation===============
IQ corr 0.4006
===============Test===============
IQ corr : 0.2095
iter  2
===============Validation===============
correlation : 0.013
max correlation index : 48
===============K-shot===============
k-shot correlation : 0.0281
k-shot max correlation index : 11
max corr pheno index 11
===============K-shot fine tuning validation===============
IQ corr -0.3404
===============Test===============
IQ corr : 0.0511
iter  3
===============Validation===============
correlation : 0.0048
max correlation index : 29
===============K-shot===============
k-shot correlation : 0.0103
k-shot max correlation index : 7
max corr pheno index 7
===============K-shot fine tuning validation===============
IQ corr 0.3498
==============

In [202]:
advanced_finetune(tbss_rd, phenotype, 50, 10)
advanced_finetune(tbss_rd, phenotype, 30, 10)
advanced_finetune(tbss_rd, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0105
max correlation index : 6
===============K-shot===============
k-shot correlation : -0.0114
k-shot max correlation index : 34
max corr pheno index 34
===============K-shot fine tuning validation===============
IQ corr -0.0999
===============Test===============
IQ corr : 0.0561
iter  2
===============Validation===============
correlation : -0.0188
max correlation index : 21
===============K-shot===============
k-shot correlation : 0.0581
k-shot max correlation index : 29
max corr pheno index 29
===============K-shot fine tuning validation===============
IQ corr -0.2608
===============Test===============
IQ corr : 0.0287
iter  3
===============Validation===============
correlation : 0.0155
max correlation index : 43
===============K-shot===============
k-shot correlation : -0.002
k-shot max correlation index : 21
max corr pheno index 21
===============K-shot fine tuning validation===============
IQ corr 0.2614
========

**Tractography**

In [203]:
advanced_finetune(tractography_ad, phenotype, 50, 10)
advanced_finetune(tractography_ad, phenotype, 30, 10)
advanced_finetune(tractography_ad, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0119
max correlation index : 15
===============K-shot===============
k-shot correlation : 0.0377
k-shot max correlation index : 34
max corr pheno index 34
===============K-shot fine tuning validation===============
IQ corr -0.0207
===============Test===============
IQ corr : -0.0535
iter  2
===============Validation===============
correlation : 0.0264
max correlation index : 22
===============K-shot===============
k-shot correlation : -0.0344
k-shot max correlation index : 21
max corr pheno index 21
===============K-shot fine tuning validation===============
IQ corr -0.0507
===============Test===============
IQ corr : -0.0341
iter  3
===============Validation===============
correlation : -0.0103
max correlation index : 9
===============K-shot===============
k-shot correlation : -0.0147
k-shot max correlation index : 48
max corr pheno index 48
===============K-shot fine tuning validation===============
IQ corr -0.2038
===

In [204]:
advanced_finetune(tractography_fa, phenotype, 50, 10)
advanced_finetune(tractography_fa, phenotype, 30, 10)
advanced_finetune(tractography_fa, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0223
max correlation index : 46
===============K-shot===============
k-shot correlation : -0.0343
k-shot max correlation index : 56
max corr pheno index 56
===============K-shot fine tuning validation===============
IQ corr 0.1405
===============Test===============
IQ corr : -0.2085
iter  2
===============Validation===============
correlation : -0.0138
max correlation index : 52
===============K-shot===============
k-shot correlation : 0.0424
k-shot max correlation index : 19
max corr pheno index 19
===============K-shot fine tuning validation===============
IQ corr 0.1294
===============Test===============
IQ corr : -0.1353
iter  3
===============Validation===============
correlation : 0.0037
max correlation index : 15
===============K-shot===============
k-shot correlation : -0.0102
k-shot max correlation index : 16
max corr pheno index 16
===============K-shot fine tuning validation===============
IQ corr -0.2801
=====

In [205]:
advanced_finetune(tractography_icvf, phenotype, 50, 10)
advanced_finetune(tractography_icvf, phenotype, 30, 10)
advanced_finetune(tractography_icvf, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0236
max correlation index : 51
===============K-shot===============
k-shot correlation : 0.0541
k-shot max correlation index : 6
max corr pheno index 6
===============K-shot fine tuning validation===============
IQ corr 0.0484
===============Test===============
IQ corr : -0.0335
iter  2
===============Validation===============
correlation : -0.0005
max correlation index : 22
===============K-shot===============
k-shot correlation : -0.079
k-shot max correlation index : 20
max corr pheno index 20
===============K-shot fine tuning validation===============
IQ corr 0.0821
===============Test===============
IQ corr : 0.0267
iter  3
===============Validation===============
correlation : 0.0296
max correlation index : 44
===============K-shot===============
k-shot correlation : -0.0266
k-shot max correlation index : 37
max corr pheno index 37
===============K-shot fine tuning validation===============
IQ corr -0.4679
========

In [206]:
advanced_finetune(tractography_isovf, phenotype, 50, 10)
advanced_finetune(tractography_isovf, phenotype, 30, 10)
advanced_finetune(tractography_isovf, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0077
max correlation index : 3
===============K-shot===============
k-shot correlation : -0.0279
k-shot max correlation index : 30
max corr pheno index 30
===============K-shot fine tuning validation===============
IQ corr -0.1249
===============Test===============
IQ corr : 0.1132
iter  2
===============Validation===============
correlation : -0.0101
max correlation index : 49
===============K-shot===============
k-shot correlation : -0.0136
k-shot max correlation index : 51
max corr pheno index 51
===============K-shot fine tuning validation===============
IQ corr 0.3875
===============Test===============
IQ corr : -0.0315
iter  3
===============Validation===============
correlation : 0.0575
max correlation index : 47
===============K-shot===============
k-shot correlation : -0.0028
k-shot max correlation index : 55
max corr pheno index 55
===============K-shot fine tuning validation===============
IQ corr -0.4725
=====

In [207]:
advanced_finetune(tractography_md, phenotype, 50, 10)
advanced_finetune(tractography_md, phenotype, 30, 10)
advanced_finetune(tractography_md, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0181
max correlation index : 56
===============K-shot===============
k-shot correlation : 0.0065
k-shot max correlation index : 51
max corr pheno index 51
===============K-shot fine tuning validation===============
IQ corr 0.4307
===============Test===============
IQ corr : 0.0546
iter  2
===============Validation===============
correlation : -0.0336
max correlation index : 19
===============K-shot===============
k-shot correlation : 0.0304
k-shot max correlation index : 51
max corr pheno index 51
===============K-shot fine tuning validation===============
IQ corr 0.2261
===============Test===============
IQ corr : 0.1706
iter  3
===============Validation===============
correlation : -0.0085
max correlation index : 23
===============K-shot===============
k-shot correlation : 0.0019
k-shot max correlation index : 23
max corr pheno index 23
===============K-shot fine tuning validation===============
IQ corr 0.2872
=========

In [208]:
advanced_finetune(tractography_od, phenotype, 50, 10)
advanced_finetune(tractography_od, phenotype, 30, 10)
advanced_finetune(tractography_od, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0057
max correlation index : 55
===============K-shot===============
k-shot correlation : 0.0012
k-shot max correlation index : 21
max corr pheno index 21
===============K-shot fine tuning validation===============
IQ corr 0.6475
===============Test===============
IQ corr : 0.1027
iter  2
===============Validation===============
correlation : 0.0215
max correlation index : 54
===============K-shot===============
k-shot correlation : 0.0399
k-shot max correlation index : 45
max corr pheno index 45
===============K-shot fine tuning validation===============
IQ corr 0.3571
===============Test===============
IQ corr : 0.0691
iter  3
===============Validation===============
correlation : -0.0034
max correlation index : 29
===============K-shot===============
k-shot correlation : 0.0254
k-shot max correlation index : 16
max corr pheno index 16
===============K-shot fine tuning validation===============
IQ corr 0.3024
==========

In [209]:
advanced_finetune(tractography_rd, phenotype, 50, 10)
advanced_finetune(tractography_rd, phenotype, 30, 10)
advanced_finetune(tractography_rd, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0156
max correlation index : 21
===============K-shot===============
k-shot correlation : 0.0117
k-shot max correlation index : 27
max corr pheno index 27
===============K-shot fine tuning validation===============
IQ corr -0.513
===============Test===============
IQ corr : -0.1009
iter  2
===============Validation===============
correlation : 0.0268
max correlation index : 36
===============K-shot===============
k-shot correlation : -0.0208
k-shot max correlation index : 51
max corr pheno index 51
===============K-shot fine tuning validation===============
IQ corr 0.2399
===============Test===============
IQ corr : 0.0816
iter  3
===============Validation===============
correlation : -0.0362
max correlation index : 37
===============K-shot===============
k-shot correlation : -0.0641
k-shot max correlation index : 23
max corr pheno index 23
===============K-shot fine tuning validation===============
IQ corr 0.1743
=======

In [210]:
advanced_finetune(tractography_fss, phenotype, 50, 10)
advanced_finetune(tractography_fss, phenotype, 30, 10)
advanced_finetune(tractography_fss, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0161
max correlation index : 52
===============K-shot===============
k-shot correlation : -0.0057
k-shot max correlation index : 37
max corr pheno index 37
===============K-shot fine tuning validation===============
IQ corr -0.3343
===============Test===============
IQ corr : 0.0372
iter  2
===============Validation===============
correlation : -0.0278
max correlation index : 28
===============K-shot===============
k-shot correlation : 0.007
k-shot max correlation index : 6
max corr pheno index 6
===============K-shot fine tuning validation===============
IQ corr 0.0757
===============Test===============
IQ corr : 0.0518
iter  3
===============Validation===============
correlation : 0.0037
max correlation index : 17
===============K-shot===============
k-shot correlation : 0.0278
k-shot max correlation index : 10
max corr pheno index 10
===============K-shot fine tuning validation===============
IQ corr -0.5989
=========

In [211]:
advanced_finetune(tractography_fssl, phenotype, 50, 10)
advanced_finetune(tractography_fssl, phenotype, 30, 10)
advanced_finetune(tractography_fssl, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0344
max correlation index : 16
===============K-shot===============
k-shot correlation : 0.0129
k-shot max correlation index : 18
max corr pheno index 18
===============K-shot fine tuning validation===============
IQ corr 0.0717
===============Test===============
IQ corr : 0.0212
iter  2
===============Validation===============
correlation : 0.0212
max correlation index : 10
===============K-shot===============
k-shot correlation : 0.0795
k-shot max correlation index : 1
max corr pheno index 1
===============K-shot fine tuning validation===============
IQ corr -0.4239
===============Test===============
IQ corr : 0.0196
iter  3
===============Validation===============
correlation : 0.0249
max correlation index : 57
===============K-shot===============
k-shot correlation : -0.0185
k-shot max correlation index : 38
max corr pheno index 38
===============K-shot fine tuning validation===============
IQ corr 0.2469
==========

**fMRI**

In [212]:
advanced_finetune(f_gamb, phenotype, 50, 10)
advanced_finetune(f_gamb, phenotype, 30, 10)
advanced_finetune(f_gamb, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0457
max correlation index : 28
===============K-shot===============
k-shot correlation : 0.0
k-shot max correlation index : 10
max corr pheno index 10
===============K-shot fine tuning validation===============
IQ corr -0.3856
===============Test===============
IQ corr : -0.0331
iter  2
===============Validation===============
correlation : -0.0157
max correlation index : 7
===============K-shot===============
k-shot correlation : -0.0553
k-shot max correlation index : 55
max corr pheno index 55
===============K-shot fine tuning validation===============
IQ corr -0.6031
===============Test===============
IQ corr : 0.0341
iter  3
===============Validation===============
correlation : -0.0498
max correlation index : 43
===============K-shot===============
k-shot correlation : -0.0379
k-shot max correlation index : 39
max corr pheno index 39
===============K-shot fine tuning validation===============
IQ corr 0.2964
=======

In [213]:
advanced_finetune(f_lang, phenotype, 50, 10)
advanced_finetune(f_lang, phenotype, 30, 10)
advanced_finetune(f_lang, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0013
max correlation index : 19
===============K-shot===============
k-shot correlation : -0.0551
k-shot max correlation index : 34
max corr pheno index 34
===============K-shot fine tuning validation===============
IQ corr 0.1736
===============Test===============
IQ corr : -0.0264
iter  2
===============Validation===============
correlation : 0.0414
max correlation index : 14
===============K-shot===============
k-shot correlation : 0.0034
k-shot max correlation index : 30
max corr pheno index 30
===============K-shot fine tuning validation===============
IQ corr 0.2047
===============Test===============
IQ corr : -0.0298
iter  3
===============Validation===============
correlation : 0.0297
max correlation index : 25
===============K-shot===============
k-shot correlation : 0.0076
k-shot max correlation index : 27
max corr pheno index 27
===============K-shot fine tuning validation===============
IQ corr 0.1106
========

In [214]:
advanced_finetune(f_motor, phenotype, 50, 10)
advanced_finetune(f_motor, phenotype, 30, 10)
advanced_finetune(f_motor, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0012
max correlation index : 27
===============K-shot===============
k-shot correlation : -0.0706
k-shot max correlation index : 18
max corr pheno index 18
===============K-shot fine tuning validation===============
IQ corr -0.1641
===============Test===============
IQ corr : 0.2485
iter  2
===============Validation===============
correlation : -0.0227
max correlation index : 0
===============K-shot===============
k-shot correlation : -0.0141
k-shot max correlation index : 21
max corr pheno index 21
===============K-shot fine tuning validation===============
IQ corr 0.1969
===============Test===============
IQ corr : -0.0333
iter  3
===============Validation===============
correlation : -0.0068
max correlation index : 28
===============K-shot===============
k-shot correlation : -0.0378
k-shot max correlation index : 40
max corr pheno index 40
===============K-shot fine tuning validation===============
IQ corr -0.0016
===

In [215]:
advanced_finetune(f_rs, phenotype, 50, 10)
advanced_finetune(f_rs, phenotype, 30, 10)
advanced_finetune(f_rs, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0283
max correlation index : 26
===============K-shot===============
k-shot correlation : 0.0551
k-shot max correlation index : 24
max corr pheno index 24
===============K-shot fine tuning validation===============
IQ corr 0.0888
===============Test===============
IQ corr : -0.0502
iter  2
===============Validation===============
correlation : -0.0444
max correlation index : 5
===============K-shot===============
k-shot correlation : 0.0199
k-shot max correlation index : 16
max corr pheno index 16
===============K-shot fine tuning validation===============
IQ corr -0.0377
===============Test===============
IQ corr : -0.1536
iter  3
===============Validation===============
correlation : -0.012
max correlation index : 18
===============K-shot===============
k-shot correlation : 0.0302
k-shot max correlation index : 9
max corr pheno index 9
===============K-shot fine tuning validation===============
IQ corr 0.0372
==========

In [216]:
advanced_finetune(f_social, phenotype, 50, 10)
advanced_finetune(f_social, phenotype, 30, 10)
advanced_finetune(f_social, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.036
max correlation index : 48
===============K-shot===============
k-shot correlation : -0.0796
k-shot max correlation index : 30
max corr pheno index 30
===============K-shot fine tuning validation===============
IQ corr 0.2879
===============Test===============
IQ corr : 0.0364
iter  2
===============Validation===============
correlation : 0.0297
max correlation index : 2
===============K-shot===============
k-shot correlation : -0.0237
k-shot max correlation index : 14
max corr pheno index 14
===============K-shot fine tuning validation===============
IQ corr -0.4678
===============Test===============
IQ corr : -0.0132
iter  3
===============Validation===============
correlation : 0.0379
max correlation index : 15
===============K-shot===============
k-shot correlation : 0.0035
k-shot max correlation index : 17
max corr pheno index 17
===============K-shot fine tuning validation===============
IQ corr -0.02
==========

In [217]:
advanced_finetune(f_wm, phenotype, 50, 10)
advanced_finetune(f_wm, phenotype, 30, 10)
advanced_finetune(f_wm, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0191
max correlation index : 25
===============K-shot===============
k-shot correlation : 0.0154
k-shot max correlation index : 37
max corr pheno index 37
===============K-shot fine tuning validation===============
IQ corr -0.029
===============Test===============
IQ corr : -0.044
iter  2
===============Validation===============
correlation : 0.0232
max correlation index : 56
===============K-shot===============
k-shot correlation : 0.0203
k-shot max correlation index : 50
max corr pheno index 50
===============K-shot fine tuning validation===============
IQ corr 0.2463
===============Test===============
IQ corr : -0.0066
iter  3
===============Validation===============
correlation : 0.0021
max correlation index : 20
===============K-shot===============
k-shot correlation : -0.0364
k-shot max correlation index : 30
max corr pheno index 30
===============K-shot fine tuning validation===============
IQ corr -0.0796
========

### Advanced - Stacking

**sMRI**

In [218]:
advanced_stacking(cort_thick, phenotype, 50, 10)
advanced_stacking(cort_thick, phenotype, 30, 10)
advanced_stacking(cort_thick, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0066
max correlation index : 21
===============Test===============
IQ corr : -0.0624
iter  2
===============Validation===============
correlation : -0.001
max correlation index : 52
===============Test===============
IQ corr : 0.0763
iter  3
===============Validation===============
correlation : 0.0141
max correlation index : 5
===============Test===============
IQ corr : -0.0752
iter  4
===============Validation===============
correlation : 0.0519
max correlation index : 3
===============Test===============
IQ corr : 0.0957
iter  5
===============Validation===============
correlation : 0.0144
max correlation index : 27
===============Test===============
IQ corr : -0.0575
iter  6
===============Validation===============
correlation : nan
max correlation index : 16
===============Test===============
IQ corr : 0.1400
iter  7
===============Validation===============
correlation : -0.0235
max correlation index : 11
==========

In [219]:
advanced_stacking(cort_area, phenotype, 50, 10)
advanced_stacking(cort_area, phenotype, 30, 10)
advanced_stacking(cort_area, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0062
max correlation index : 45
===============Test===============
IQ corr : -0.0015
iter  2
===============Validation===============
correlation : -0.0232
max correlation index : 13
===============Test===============
IQ corr : 0.0384
iter  3
===============Validation===============
correlation : 0.0257
max correlation index : 48
===============Test===============
IQ corr : -0.0301
iter  4
===============Validation===============
correlation : -0.0099
max correlation index : 42
===============Test===============
IQ corr : -0.0416
iter  5
===============Validation===============
correlation : -0.0194
max correlation index : 1
===============Test===============
IQ corr : 0.1896
iter  6
===============Validation===============
correlation : -0.0002
max correlation index : 30
===============Test===============
IQ corr : 0.0870
iter  7
===============Validation===============
correlation : -0.0219
max correlation index : 18
=

In [220]:
advanced_stacking(cort_vol, phenotype, 50, 10)
advanced_stacking(cort_vol, phenotype, 30, 10)
advanced_stacking(cort_vol, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0083
max correlation index : 33
===============Test===============
IQ corr : -0.0820
iter  2
===============Validation===============
correlation : -0.0176
max correlation index : 49
===============Test===============
IQ corr : 0.1049
iter  3
===============Validation===============
correlation : -0.0023
max correlation index : 51
===============Test===============
IQ corr : 0.1400
iter  4
===============Validation===============
correlation : -0.0029
max correlation index : 4
===============Test===============
IQ corr : -0.0033
iter  5
===============Validation===============
correlation : -0.0077
max correlation index : 25
===============Test===============
IQ corr : -0.1435
iter  6
===============Validation===============
correlation : -0.0033
max correlation index : 14
===============Test===============
IQ corr : 0.0369
iter  7
===============Validation===============
correlation : 0.0486
max correlation index : 15
=

**TBSS**

In [221]:
advanced_stacking(tbss_ad, phenotype, 50, 10)
advanced_stacking(tbss_ad, phenotype, 30, 10)
advanced_stacking(tbss_ad, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0058
max correlation index : 22
===============Test===============
IQ corr : 0.0577
iter  2
===============Validation===============
correlation : -0.0247
max correlation index : 57
===============Test===============
IQ corr : 0.1769
iter  3
===============Validation===============
correlation : 0.0089
max correlation index : 2
===============Test===============
IQ corr : 0.0384
iter  4
===============Validation===============
correlation : 0.0042
max correlation index : 44
===============Test===============
IQ corr : -0.0371
iter  5
===============Validation===============
correlation : 0.0072
max correlation index : 19
===============Test===============
IQ corr : 0.1140
iter  6
===============Validation===============
correlation : -0.0261
max correlation index : 17
===============Test===============
IQ corr : 0.1618
iter  7
===============Validation===============
correlation : 0.0192
max correlation index : 12
======

In [222]:
advanced_stacking(tbss_fa, phenotype, 50, 10)
advanced_stacking(tbss_fa, phenotype, 30, 10)
advanced_stacking(tbss_fa, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0132
max correlation index : 42
===============Test===============
IQ corr : -0.1023
iter  2
===============Validation===============
correlation : -0.028
max correlation index : 11
===============Test===============
IQ corr : -0.0405
iter  3
===============Validation===============
correlation : 0.054
max correlation index : 42
===============Test===============
IQ corr : 0.0185
iter  4
===============Validation===============
correlation : -0.009
max correlation index : 22
===============Test===============
IQ corr : 0.0024
iter  5
===============Validation===============
correlation : 0.035
max correlation index : 36
===============Test===============
IQ corr : -0.0070
iter  6
===============Validation===============
correlation : -0.0177
max correlation index : 41
===============Test===============
IQ corr : 0.0946
iter  7
===============Validation===============
correlation : 0.0128
max correlation index : 37
======

In [223]:
advanced_stacking(tbss_icvf, phenotype, 50, 10)
advanced_stacking(tbss_icvf, phenotype, 30, 10)
advanced_stacking(tbss_icvf, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0322
max correlation index : 11
===============Test===============
IQ corr : 0.1559
iter  2
===============Validation===============
correlation : 0.0179
max correlation index : 13
===============Test===============
IQ corr : 0.0221
iter  3
===============Validation===============
correlation : 0.0273
max correlation index : 17
===============Test===============
IQ corr : 0.1313
iter  4
===============Validation===============
correlation : 0.0525
max correlation index : 40
===============Test===============
IQ corr : -0.0299
iter  5
===============Validation===============
correlation : 0.0272
max correlation index : 34
===============Test===============
IQ corr : 0.2490
iter  6
===============Validation===============
correlation : 0.0004
max correlation index : 55
===============Test===============
IQ corr : -0.1175
iter  7
===============Validation===============
correlation : -0.0112
max correlation index : 20
=====

In [224]:
advanced_stacking(tbss_isovf, phenotype, 50, 10)
advanced_stacking(tbss_isovf, phenotype, 30, 10)
advanced_stacking(tbss_isovf, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0079
max correlation index : 8
===============Test===============
IQ corr : -0.1753
iter  2
===============Validation===============
correlation : 0.0475
max correlation index : 2
===============Test===============
IQ corr : -0.0356
iter  3
===============Validation===============
correlation : 0.0061
max correlation index : 12
===============Test===============
IQ corr : 0.0549
iter  4
===============Validation===============
correlation : 0.015
max correlation index : 31
===============Test===============
IQ corr : 0.1495
iter  5
===============Validation===============
correlation : -0.0184
max correlation index : 16
===============Test===============
IQ corr : 0.1325
iter  6
===============Validation===============
correlation : -0.0013
max correlation index : 55
===============Test===============
IQ corr : -0.0267
iter  7
===============Validation===============
correlation : -0.0104
max correlation index : 18
======

In [225]:
advanced_stacking(tbss_md, phenotype, 50, 10)
advanced_stacking(tbss_md, phenotype, 30, 10)
advanced_stacking(tbss_md, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0537
max correlation index : 25
===============Test===============
IQ corr : 0.0450
iter  2
===============Validation===============
correlation : 0.0227
max correlation index : 47
===============Test===============
IQ corr : 0.0763
iter  3
===============Validation===============
correlation : 0.0025
max correlation index : 31
===============Test===============
IQ corr : 0.0036
iter  4
===============Validation===============
correlation : 0.0375
max correlation index : 9
===============Test===============
IQ corr : 0.0160
iter  5
===============Validation===============
correlation : 0.0477
max correlation index : 44
===============Test===============
IQ corr : 0.1409
iter  6
===============Validation===============
correlation : 0.0018
max correlation index : 15
===============Test===============
IQ corr : -0.0059
iter  7
===============Validation===============
correlation : 0.0071
max correlation index : 42
=========

In [226]:
advanced_stacking(tbss_od, phenotype, 50, 10)
advanced_stacking(tbss_od, phenotype, 30, 10)
advanced_stacking(tbss_od, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.044
max correlation index : 39
===============Test===============
IQ corr : 0.0501
iter  2
===============Validation===============
correlation : -0.0137
max correlation index : 14
===============Test===============
IQ corr : -0.2595
iter  3
===============Validation===============
correlation : -0.0087
max correlation index : 9
===============Test===============
IQ corr : 0.0893
iter  4
===============Validation===============
correlation : 0.0179
max correlation index : 57
===============Test===============
IQ corr : 0.2067
iter  5
===============Validation===============
correlation : 0.0047
max correlation index : 23
===============Test===============
IQ corr : -0.1186
iter  6
===============Validation===============
correlation : -0.0136
max correlation index : 49
===============Test===============
IQ corr : -0.0200
iter  7
===============Validation===============
correlation : 0.0376
max correlation index : 36
====

In [227]:
advanced_stacking(tbss_rd, phenotype, 50, 10)
advanced_stacking(tbss_rd, phenotype, 30, 10)
advanced_stacking(tbss_rd, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0089
max correlation index : 26
===============Test===============
IQ corr : -0.0387
iter  2
===============Validation===============
correlation : 0.0016
max correlation index : 10
===============Test===============
IQ corr : -0.1561
iter  3
===============Validation===============
correlation : -0.0175
max correlation index : 17
===============Test===============
IQ corr : -0.1068
iter  4
===============Validation===============
correlation : -0.0633
max correlation index : 28
===============Test===============
IQ corr : -0.0278
iter  5
===============Validation===============
correlation : 0.0129
max correlation index : 17
===============Test===============
IQ corr : -0.0387
iter  6
===============Validation===============
correlation : 0.0262
max correlation index : 25
===============Test===============
IQ corr : -0.0280
iter  7
===============Validation===============
correlation : -0.0207
max correlation index : 52

**Tractography**

In [228]:
advanced_stacking(tractography_ad, phenotype, 50, 10)
advanced_stacking(tractography_ad, phenotype, 30, 10)
advanced_stacking(tractography_ad, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0067
max correlation index : 27
===============Test===============
IQ corr : 0.0622
iter  2
===============Validation===============
correlation : 0.0043
max correlation index : 3
===============Test===============
IQ corr : -0.0769
iter  3
===============Validation===============
correlation : 0.0096
max correlation index : 21
===============Test===============
IQ corr : 0.0046
iter  4
===============Validation===============
correlation : 0.0127
max correlation index : 11
===============Test===============
IQ corr : 0.0791
iter  5
===============Validation===============
correlation : 0.028
max correlation index : 9
===============Test===============
IQ corr : -0.0268
iter  6
===============Validation===============
correlation : -0.0548
max correlation index : 23
===============Test===============
IQ corr : -0.0824
iter  7
===============Validation===============
correlation : 0.0045
max correlation index : 40
=======

In [229]:
advanced_stacking(tractography_fa, phenotype, 50, 10)
advanced_stacking(tractography_fa, phenotype, 30, 10)
advanced_stacking(tractography_fa, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0011
max correlation index : 54
===============Test===============
IQ corr : 0.0762
iter  2
===============Validation===============
correlation : 0.0187
max correlation index : 37
===============Test===============
IQ corr : -0.0387
iter  3
===============Validation===============
correlation : -0.0181
max correlation index : 1
===============Test===============
IQ corr : -0.1314
iter  4
===============Validation===============
correlation : -0.0028
max correlation index : 21
===============Test===============
IQ corr : -0.2255
iter  5
===============Validation===============
correlation : -0.0316
max correlation index : 9
===============Test===============
IQ corr : 0.0657
iter  6
===============Validation===============
correlation : 0.0172
max correlation index : 25
===============Test===============
IQ corr : 0.0608
iter  7
===============Validation===============
correlation : 0.0086
max correlation index : 19
====

In [230]:
advanced_stacking(tractography_icvf, phenotype, 50, 10)
advanced_stacking(tractography_icvf, phenotype, 30, 10)
advanced_stacking(tractography_icvf, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.042
max correlation index : 25
===============Test===============
IQ corr : -0.0452
iter  2
===============Validation===============
correlation : -0.0133
max correlation index : 33
===============Test===============
IQ corr : 0.0081
iter  3
===============Validation===============
correlation : -0.004
max correlation index : 11
===============Test===============
IQ corr : -0.0814
iter  4
===============Validation===============
correlation : -0.0289
max correlation index : 19
===============Test===============
IQ corr : -0.1096
iter  5
===============Validation===============
correlation : 0.0057
max correlation index : 25
===============Test===============
IQ corr : -0.0864
iter  6
===============Validation===============
correlation : -0.0353
max correlation index : 27
===============Test===============
IQ corr : 0.0307
iter  7
===============Validation===============
correlation : -0.0122
max correlation index : 52
==

In [231]:
advanced_stacking(tractography_isovf, phenotype, 50, 10)
advanced_stacking(tractography_isovf, phenotype, 30, 10)
advanced_stacking(tractography_isovf, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0222
max correlation index : 9
===============Test===============
IQ corr : 0.0885
iter  2
===============Validation===============
correlation : -0.0307
max correlation index : 39
===============Test===============
IQ corr : 0.1102
iter  3
===============Validation===============
correlation : 0.0216
max correlation index : 56
===============Test===============
IQ corr : 0.0689
iter  4
===============Validation===============
correlation : -0.0262
max correlation index : 36
===============Test===============
IQ corr : 0.0177
iter  5
===============Validation===============
correlation : -0.036
max correlation index : 17
===============Test===============
IQ corr : -0.0243
iter  6
===============Validation===============
correlation : 0.0027
max correlation index : 14
===============Test===============
IQ corr : -0.1752
iter  7
===============Validation===============
correlation : 0.0063
max correlation index : 37
=====

In [232]:
advanced_stacking(tractography_md, phenotype, 50, 10)
advanced_stacking(tractography_md, phenotype, 30, 10)
advanced_stacking(tractography_md, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0173
max correlation index : 44
===============Test===============
IQ corr : 0.1331
iter  2
===============Validation===============
correlation : 0.0032
max correlation index : 18
===============Test===============
IQ corr : -0.2912
iter  3
===============Validation===============
correlation : 0.0033
max correlation index : 48
===============Test===============
IQ corr : -0.0761
iter  4
===============Validation===============
correlation : -0.0161
max correlation index : 54
===============Test===============
IQ corr : -0.0230
iter  5
===============Validation===============
correlation : 0.0037
max correlation index : 1
===============Test===============
IQ corr : -0.0665
iter  6
===============Validation===============
correlation : -0.0206
max correlation index : 25
===============Test===============
IQ corr : 0.0399
iter  7
===============Validation===============
correlation : 0.0275
max correlation index : 1
=====

In [233]:
advanced_stacking(tractography_od, phenotype, 50, 10)
advanced_stacking(tractography_od, phenotype, 30, 10)
advanced_stacking(tractography_od, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0284
max correlation index : 16
===============Test===============
IQ corr : -0.0539
iter  2
===============Validation===============
correlation : 0.0068
max correlation index : 33
===============Test===============
IQ corr : -0.0135
iter  3
===============Validation===============
correlation : 0.0161
max correlation index : 8
===============Test===============
IQ corr : -0.0819
iter  4
===============Validation===============
correlation : 0.0019
max correlation index : 8
===============Test===============
IQ corr : -0.0657
iter  5
===============Validation===============
correlation : -0.0131
max correlation index : 10
===============Test===============
IQ corr : -0.1799
iter  6
===============Validation===============
correlation : 0.0114
max correlation index : 38
===============Test===============
IQ corr : -0.0078
iter  7
===============Validation===============
correlation : -0.0105
max correlation index : 16
===

In [234]:
advanced_stacking(tractography_rd, phenotype, 50, 10)
advanced_stacking(tractography_rd, phenotype, 30, 10)
advanced_stacking(tractography_rd, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0111
max correlation index : 15
===============Test===============
IQ corr : 0.1143
iter  2
===============Validation===============
correlation : 0.0389
max correlation index : 37
===============Test===============
IQ corr : -0.0147
iter  3
===============Validation===============
correlation : 0.0108
max correlation index : 21
===============Test===============
IQ corr : -0.1124
iter  4
===============Validation===============
correlation : 0.0112
max correlation index : 50
===============Test===============
IQ corr : 0.0166
iter  5
===============Validation===============
correlation : 0.0179
max correlation index : 27
===============Test===============
IQ corr : 0.1012
iter  6
===============Validation===============
correlation : 0.0514
max correlation index : 6
===============Test===============
IQ corr : -0.1104
iter  7
===============Validation===============
correlation : -0.0252
max correlation index : 24
=====

In [235]:
advanced_stacking(tractography_fss, phenotype, 50, 10)
advanced_stacking(tractography_fss, phenotype, 30, 10)
advanced_stacking(tractography_fss, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0051
max correlation index : 31
===============Test===============
IQ corr : 0.0187
iter  2
===============Validation===============
correlation : -0.0016
max correlation index : 37
===============Test===============
IQ corr : -0.2683
iter  3
===============Validation===============
correlation : -0.0221
max correlation index : 12
===============Test===============
IQ corr : 0.1101
iter  4
===============Validation===============
correlation : -0.0001
max correlation index : 44
===============Test===============
IQ corr : -0.0182
iter  5
===============Validation===============
correlation : -0.0007
max correlation index : 57
===============Test===============
IQ corr : -0.2099
iter  6
===============Validation===============
correlation : 0.0286
max correlation index : 53
===============Test===============
IQ corr : 0.1309
iter  7
===============Validation===============
correlation : 0.0427
max correlation index : 20
=

In [236]:
advanced_stacking(tractography_fssl, phenotype, 50, 10)
advanced_stacking(tractography_fssl, phenotype, 30, 10)
advanced_stacking(tractography_fssl, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0002
max correlation index : 22
===============Test===============
IQ corr : 0.0454
iter  2
===============Validation===============
correlation : -0.0543
max correlation index : 20
===============Test===============
IQ corr : -0.0589
iter  3
===============Validation===============
correlation : 0.0503
max correlation index : 31
===============Test===============
IQ corr : 0.0222
iter  4
===============Validation===============
correlation : 0.0372
max correlation index : 30
===============Test===============
IQ corr : -0.0395
iter  5
===============Validation===============
correlation : 0.0165
max correlation index : 3
===============Test===============
IQ corr : 0.0889
iter  6
===============Validation===============
correlation : -0.0173
max correlation index : 26
===============Test===============
IQ corr : 0.0672
iter  7
===============Validation===============
correlation : -0.0155
max correlation index : 24
====

**fMRI**

In [237]:
advanced_stacking(f_gamb, phenotype, 50, 10)
advanced_stacking(f_gamb, phenotype, 30, 10)
advanced_stacking(f_gamb, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.022
max correlation index : 37
===============Test===============
IQ corr : 0.0502
iter  2
===============Validation===============
correlation : -0.007
max correlation index : 0
===============Test===============
IQ corr : -0.1202
iter  3
===============Validation===============
correlation : 0.0094
max correlation index : 3
===============Test===============
IQ corr : 0.1875
iter  4
===============Validation===============
correlation : -0.007
max correlation index : 11
===============Test===============
IQ corr : -0.1836
iter  5
===============Validation===============
correlation : 0.0161
max correlation index : 3
===============Test===============
IQ corr : -0.0783
iter  6
===============Validation===============
correlation : 0.0098
max correlation index : 31
===============Test===============
IQ corr : -0.0508
iter  7
===============Validation===============
correlation : 0.0458
max correlation index : 35
========

In [238]:
advanced_stacking(f_lang, phenotype, 50, 10)
advanced_stacking(f_lang, phenotype, 30, 10)
advanced_stacking(f_lang, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0306
max correlation index : 24
===============Test===============
IQ corr : 0.0964
iter  2
===============Validation===============
correlation : -0.0027
max correlation index : 28
===============Test===============
IQ corr : -0.0804
iter  3
===============Validation===============
correlation : -0.0132
max correlation index : 31
===============Test===============
IQ corr : 0.1933
iter  4
===============Validation===============
correlation : -0.0292
max correlation index : 12
===============Test===============
IQ corr : -0.0646
iter  5
===============Validation===============
correlation : -0.0195
max correlation index : 19
===============Test===============
IQ corr : -0.1115
iter  6
===============Validation===============
correlation : 0.0386
max correlation index : 25
===============Test===============
IQ corr : -0.1058
iter  7
===============Validation===============
correlation : 0.0109
max correlation index : 43


In [239]:
advanced_stacking(f_motor, phenotype, 50, 10)
advanced_stacking(f_motor, phenotype, 30, 10)
advanced_stacking(f_motor, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0149
max correlation index : 19
===============Test===============
IQ corr : 0.0426
iter  2
===============Validation===============
correlation : -0.0089
max correlation index : 48
===============Test===============
IQ corr : -0.0025
iter  3
===============Validation===============
correlation : 0.02
max correlation index : 56
===============Test===============
IQ corr : -0.0920
iter  4
===============Validation===============
correlation : 0.0113
max correlation index : 22
===============Test===============
IQ corr : 0.0687
iter  5
===============Validation===============
correlation : 0.0234
max correlation index : 39
===============Test===============
IQ corr : -0.0412
iter  6
===============Validation===============
correlation : 0.0288
max correlation index : 52
===============Test===============
IQ corr : -0.1565
iter  7
===============Validation===============
correlation : -0.0133
max correlation index : 45
====

In [240]:
advanced_stacking(f_rs, phenotype, 50, 10)
advanced_stacking(f_rs, phenotype, 30, 10)
advanced_stacking(f_rs, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0015
max correlation index : 1
===============Test===============
IQ corr : 0.0392
iter  2
===============Validation===============
correlation : nan
max correlation index : 30
===============Test===============
IQ corr : 0.0653
iter  3
===============Validation===============
correlation : 0.0277
max correlation index : 24
===============Test===============
IQ corr : 0.0786
iter  4
===============Validation===============
correlation : 0.0215
max correlation index : 43
===============Test===============
IQ corr : -0.1846
iter  5
===============Validation===============
correlation : 0.0144
max correlation index : 38
===============Test===============
IQ corr : -0.0895
iter  6
===============Validation===============
correlation : -0.043
max correlation index : 1
===============Test===============
IQ corr : 0.1768
iter  7
===============Validation===============
correlation : -0.0146
max correlation index : 21
==========

In [241]:
advanced_stacking(f_social, phenotype, 50, 10)
advanced_stacking(f_social, phenotype, 30, 10)
advanced_stacking(f_social, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : -0.0031
max correlation index : 44
===============Test===============
IQ corr : 0.0526
iter  2
===============Validation===============
correlation : -0.0185
max correlation index : 35
===============Test===============
IQ corr : 0.0256
iter  3
===============Validation===============
correlation : 0.0052
max correlation index : 38
===============Test===============
IQ corr : -0.1056
iter  4
===============Validation===============
correlation : 0.018
max correlation index : 20
===============Test===============
IQ corr : -0.1341
iter  5
===============Validation===============
correlation : -0.006
max correlation index : 51
===============Test===============
IQ corr : 0.1375
iter  6
===============Validation===============
correlation : -0.0121
max correlation index : 40
===============Test===============
IQ corr : -0.2009
iter  7
===============Validation===============
correlation : -0.0281
max correlation index : 47
===

In [242]:
advanced_stacking(f_wm, phenotype, 50, 10)
advanced_stacking(f_wm, phenotype, 30, 10)
advanced_stacking(f_wm, phenotype, 10, 10)

iter  1
===============Validation===============
correlation : 0.0206
max correlation index : 3
===============Test===============
IQ corr : 0.0821
iter  2
===============Validation===============
correlation : -0.0125
max correlation index : 14
===============Test===============
IQ corr : -0.0860
iter  3
===============Validation===============
correlation : -0.0131
max correlation index : 17
===============Test===============
IQ corr : -0.0434
iter  4
===============Validation===============
correlation : -0.0142
max correlation index : 14
===============Test===============
IQ corr : -0.0527
iter  5
===============Validation===============
correlation : 0.004
max correlation index : 36
===============Test===============
IQ corr : 0.0215
iter  6
===============Validation===============
correlation : 0.0155
max correlation index : 57
===============Test===============
IQ corr : 0.1103
iter  7
===============Validation===============
correlation : 0.0074
max correlation index : 9
======